In [ ]:
# The below is the scripting process that defines the dairy land footprint, runs LivSim, and calculates the GHG emissions
# from the base and mitigation scenarios described in the article. 


def model_in(data_wb,scenario_parameters,l,s,timestep,MC_sims,breed,cohort,subsectors,herd_size,herd_size_y_prev,herd_size_base,check_feasibility):


    # Load data sheets from excel
    ghg_sheet= data_wb.sheet_by_index(1)
    uncertainty_sheet= data_wb.sheet_by_index(2)
       
    if (l == 'MRT'):
        sheet = data_wb.sheet_by_index(3)
        diet_data=(cwd+str('\\diet_MRT.xlsx'))
        
    elif (l == 'MRH'):
        sheet = data_wb.sheet_by_index(4) 
        diet_data=(cwd+str('\\diet_MRH.xlsx'))
        
    yield_sheet=data_wb.sheet_by_index(5)
    TLU_dat=data_wb.sheet_by_index(6)
    feed_util_effcy=data_wb.sheet_by_index(8)
    diet_wb = xlrd.open_workbook(diet_data)

    
# DEFINE DATA PARAMETERS/VARIABLES USED IN SIMULATIONS #

    # milk yield
    milk_yield_reg = {} 
    milk_yield_lact={}
    
    # feed intake dictionaries
    grass_intake={}
    stover_intake={}
    maize_bran_intake={}
    sfsm_intake={}
    napier_intake={}
    napier_hy_intake={}
    pasture_intake={}
    
    # dietary and excretion variables
    faecal_Nitrogen = {}
    urinary_Nitrogen = {}
    fraction_roughage_eaten = {}
    fraction_conc_eaten={}
    gross_energy = {}
    crude_protein_intake={}
    DMI={}
    DMD={}
    acid_detergent_fibre={}
    neutral_detergent_fibre={}
    Diet_N={}
    ADF={}
    NDF={}
    ME={}
    Ym={}
    VS={}
    
    TLU_equiv={}
    utn_efy={}
    
    # GHG emissions results
    enteric_CH4={}
    manure_CH4={}
    
    soil_N2O_cropland={}
    soil_N2O_direct_cropland={}
    soil_N2O_indirect_cropland={}
    soil_N2O_indirect_vol_cropland={}
    soil_N2O_indirect_leach_cropland={}
    
    soil_N2O_grassland={}
    soil_N2O_direct_grassland={}
    soil_N2O_indirect_grassland={}
    soil_N2O_indirect_vol_grassland={}
    soil_N2O_indirect_leach_grassland={}

    manure_N2O={}
    manure_N2O_direct={}
    manure_N2O_vol={}
    manure_N2O_leach={}

    soil_N2O_feed={}
    soil_N2O_feed_direct={}
    soil_N2O_feed_adn={}
    soil_N2O_feed_lch={}
    
    energy_use_CO2={}
    
    # as intensities  
    enteric_CH4_intensity={}
    manure_CH4_intensity={}
    soil_N2O_intensity={}
    manure_N2O_intensity={}
    direct_emission_intensity={}
    

    # parameters and results that are random (for uncertainty analysis)
    Ym_rd={}
    B0_rd={}
    MCF_rd={}
    EF_3_FAECAL_rd={}
    EF_3_URINARY_rd={}
     
    enteric_CH4_rd={}
    manure_CH4_rd={}
    manure_N2O_rd={}
    manure_N2O_direct_rd={}
    manure_N2O_leach_rd={}
    manure_N2O_vol_rd={}
    energy_use_CO2_rd={}
    
    soil_N2O_feed_direct_rd={}
    soil_N2O_feed_adn_rd={}
    soil_N2O_feed_lch_rd={}
    soil_N2O_feed_rd={}
    soil_N2O_rd={}
    
    grassland_area_expansion_emissions_rd={}
    cropland_area_expansion_emissions_rd={}

    herd_size_rd={}
    direct_emission_intensity_rd={}
    direct_emission_absolute_rd={}
    
    crop_primary_area_rd={}
    feed_primary_area_rd={}
    
    grassland_area_expansion_emissions_rd_ss={}
    cropland_area_expansion_emissions_rd_ss={}
    
    for r in range(0,MC_sims):
        grassland_area_expansion_emissions_rd[r]=0
        cropland_area_expansion_emissions_rd[r]=0
        for ss in subsectors:
            grassland_area_expansion_emissions_rd_ss[r,ss]=0
            cropland_area_expansion_emissions_rd_ss[r,ss]=0
    
    RESULTS={} # dictionary to store results of the given iteration
    
    # land footprint dictionaries
    feed_area_base={}
    feed_primary_area_base={}
    
    feed_area={}
    feed_primary_area={}
    
    total_grasslands_area={}
    
    crop_primary_area={}
    crop_primary_area_base={}
    
    pasture_area={}
    pasture_area_base={}
    
    grass_area={}
    grass_area_base={}
    
    stover_area={}
    stover_area_base={}
    
    napier_area={}
    napier_area_base={}
    
    napier_hy_area={}
    napier_hy_area_base={}
    
    maize_as_bran_area={}
    maize_as_bran_area_base={}
    
    sunflower_as_meal_area={}
    sunflower_as_meal_area_base={}
    
    # monthly feed parameter dictionaries
    mo_me_short={}
    mo_mp_short={}
    
    mo_dp_cp ={}
    mo_dp_me ={}
    
    mo_fi_nhy = {}
    mo_fi_np={}
    
    mo_fi_ns = {}
    mo_fi_sto={}
    mo_fi_mb = {}
    mo_fi_sc={}
    mo_fi_grs = {}
    mo_fi_pt={}
    
    
    count_calendar_month={} # dictionary to keep track of calendar months of year to account for seasonality
    
    # Populate Parameters
    
    # Emission factors
    if (l == 'MRT'):
        col_ghg=2
    elif (l=='MRH'):
        col_ghg=3
            
    # manure EFs
    B0=ghg_sheet.cell_value(2,col_ghg)
    MCF_LOC=ghg_sheet.cell_value(3,col_ghg)
    MCF_IMP=ghg_sheet.cell_value(4,col_ghg)
    EF_1=ghg_sheet.cell_value(5,col_ghg)
    EF_3_FAECAL=ghg_sheet.cell_value(6,col_ghg)
    EF_3_URINARY=ghg_sheet.cell_value(7,col_ghg)
    EF_3_PRP=ghg_sheet.cell_value(8,col_ghg)
    FRAC_MANURE_PRP_LOC=ghg_sheet.cell_value(9,col_ghg)
    FRAC_MANURE_MMT_LOC=ghg_sheet.cell_value(10,col_ghg)
    FRAC_MANURE_PRP_IMP=ghg_sheet.cell_value(11,col_ghg)
    FRAC_MANURE_MMT_IMP=ghg_sheet.cell_value(12,col_ghg)
    EF_4_LOC=ghg_sheet.cell_value(13,col_ghg)
    EF_4_IMP=ghg_sheet.cell_value(14,col_ghg)
    EF_5_LOC=ghg_sheet.cell_value(15,col_ghg)
    EF_5_IMP=ghg_sheet.cell_value(16,col_ghg)
    FRAC_vol_LOC=ghg_sheet.cell_value(17,col_ghg)
    FRAC_vol_IMP=ghg_sheet.cell_value(18,col_ghg)
    FRAC_leach_LOC=ghg_sheet.cell_value(19,col_ghg)
    FRAC_leach_IMP=ghg_sheet.cell_value(20,col_ghg)

    # soil related EFs
    EF_4_adn=ghg_sheet.cell_value(21,col_ghg)
    EF_5_lch=ghg_sheet.cell_value(22,col_ghg)
    FRAC_gasf=ghg_sheet.cell_value(23,col_ghg)
    FRAC_gasm=ghg_sheet.cell_value(24,col_ghg)
    FRAC_loss_MMS=ghg_sheet.cell_value(25,col_ghg)
   
    # land C stock densities
    mean_C_density=ghg_sheet.cell_value(26,col_ghg)
    Maximal_Forest_C_Density=ghg_sheet.cell_value(27,col_ghg)
    MEAN_C_DENSITY_GRASSLAND=ghg_sheet.cell_value(28,col_ghg)
    luc_emission_coefficient_grass_to_crop= 11 # as described in article
   
    luc_emission_coefficient_total_area=mean_C_density-MEAN_C_DENSITY_GRASSLAND
    
    # period over which LUC emissions are amortized
    amort_period=ghg_sheet.cell_value(29,col_ghg)
    
    ## allocation factors
    allocation_factor_grass=ghg_sheet.cell_value(30,col_ghg)
    allocation_factor_napier=ghg_sheet.cell_value(31,col_ghg)
    allocation_factor_maize_bran=ghg_sheet.cell_value(32,col_ghg)
    allocation_factor_sunflower_seed_meal=ghg_sheet.cell_value(33,col_ghg)
    allocation_factor_stover=ghg_sheet.cell_value(34,col_ghg)
    
    # Energy related emissions 
    feed_embodied_CO2={"maize_bran": ghg_sheet.cell_value(35,col_ghg),"sunflower_seed_meal":ghg_sheet.cell_value(35,col_ghg)} # kg co2eq per Mg 
    fertilizer_embodied_CO2=ghg_sheet.cell_value(36,col_ghg)
        
        
    #  livestock weights in tlu
    TLU_equiv['local','cow']=TLU_dat.cell_value(1, 2)
    TLU_equiv['improved','cow']=TLU_dat.cell_value(6, 2)
    
    TLU_equiv['local','heifer']=TLU_dat.cell_value(2, 2)
    TLU_equiv['improved','heifer']=TLU_dat.cell_value(7, 2)
    
    TLU_equiv['local','bull']=TLU_dat.cell_value(4, 2)
    TLU_equiv['improved','bull']=TLU_dat.cell_value(9, 2)
    
    TLU_equiv['local','fe_calf']=TLU_dat.cell_value(3, 2)
    TLU_equiv['improved','fe_calf']=TLU_dat.cell_value(9, 2)
    
    TLU_equiv['local','ml_calf']=TLU_dat.cell_value(3, 2)
    TLU_equiv['improved','ml_calf']=TLU_dat.cell_value(9, 2)
    
    TLU_equiv['local','juv_male']=TLU_dat.cell_value(5, 2)
    TLU_equiv['improved','juv_male']=TLU_dat.cell_value(10, 2)
    
    # utilization efficiencies of biomass sources
    utn_efy[('grass')]=feed_util_effcy.cell_value(1,1)
    utn_efy[('pasture')]=feed_util_effcy.cell_value(2,1)
    utn_efy[('napier')]=feed_util_effcy.cell_value(3,1)
    utn_efy[('napier_hy')]=feed_util_effcy.cell_value(4,1)
    utn_efy[('maize_bran')]=feed_util_effcy.cell_value(5,1)
    utn_efy[('sunflower_seed_meal')]=feed_util_effcy.cell_value(6,1)
    utn_efy[('stover')]=feed_util_effcy.cell_value(7,1)

    # DEFINE AND INITIALIZE RESULTS DATAFRAME VARIABLES
    RESULTS['v1_2_Milk_yield_total']=0
    RESULTS['v1_2_Milk_yield_base_year_kg_yr']=0

    RESULTS['v6_1_Enteric_CH4_aggregate_Mg_CO2eq_per_lps']=0
    RESULTS['v6_2_Manure_CH4_aggregate_Mg_CO2eq_per_lps']=0
    RESULTS['v6_3_Manure_N2O_aggregate_Mg_CO2eq_per_lps']=0
    RESULTS['v6_4_Soil_N2O_cropland_aggregate_Mg_CO2eq_per_lps']=0
    RESULTS['v6_4_Soil_N2O_grassland_aggregate_Mg_CO2eq_per_lps']=0
    RESULTS['v6_4_2_Feed_CO2_aggregate_Mg_CO2eq_per_lps']=0
    RESULTS['v6_5_Direct_emissions_aggregate_Mg_CO2eq_per_lps']=0
    RESULTS['v6_6_grassland_expansion_emissions_Mg_CO2eq_per_lps']=0
    RESULTS['v6_8_cropland_expansion_emissions_Mg_CO2eq_per_lps']=0
   
    RESULTS['v9_2_1_Grasslands_area_ha_per_lps'] =0
    RESULTS['v9_2_2_Feed_area_ha_per_lps'] =0
    RESULTS['v9_2_3_Crop_primary_area_ha_per_lps']=0

    RESULTS['v3_2_1_Feed_intake_grass_%_dm']=0
    RESULTS['v3_2_2_Feed_intake_stover_%_dm']=0
    RESULTS['v3_2_3_Feed_intake_napier_%_dm']=0
    RESULTS['v3_2_4_Feed_intake_maize_bran_%_dm']=0
    RESULTS['v3_2_5_Feed_intake_sfsm_%_dm']=0   

    RESULTS['v3_2_6_Feed_intake_pasture_%_dm']=0
    RESULTS['v3_2_6_Feed_intake_grass_kg_dm']=0
    RESULTS['v3_2_7_Feed_intake_stover_kg_dm']=0
    RESULTS['v3_2_8_Feed_intake_napier_kg_dm']=0
    RESULTS['v3_2_8_Feed_intake_napier_hy_kg_dm']=0
    RESULTS['v3_2_9_Feed_intake_maize_bran_kg_dm']=0
    RESULTS['v3_2_10_Feed_intake_sfsm_kg_dm']=0
    RESULTS['v3_2_11_Feed_intake_pasture_kg_dm']=0
    
    RESULTS['v5_6_Total_feed_primary_area_expansion']=0
                         
    RESULTS['v9_15_Grass_land_use_ha_per_TLU']=0
    RESULTS['v9_16_Stover_land_use_ha_per_TLU']=0
    RESULTS['v9_17_Napier_land_use_ha_per_TLU']=0
    RESULTS['v9_17_Napier_hy_land_use_ha_per_TLU']=0
    RESULTS['v9_18_Sunflower_land_use_ha_per_TLU']=0
    RESULTS['v9_19_Maize_land_use_ha_per_TLU']=0
    RESULTS['v9_20_Pasture_land_use_ha_per_TLU']=0
    
    RESULTS['v8_4_Herd_size_total_1000_head_per_lps']=0
    RESULTS['v8_6_Herd_size_total_TLU_per_lps']=0
    
    RESULTS['v6_0_Units_limited']=0

    RESULTS['v9_4_1_Grass_area_ha_per_lps']=0
    RESULTS['v9_4_2_Stover_area_ha_per_lps']=0
    RESULTS['v9_4_3_Napier_area_ha_per_lps']=0
    RESULTS['v9_4_4_Napier_hy_area_ha_per_lps']=0
    RESULTS['v9_4_5_Sunflower_area_ha_per_lps']=0
    RESULTS['v9_4_6_Maize_area_ha_per_lps']=0
    RESULTS['v9_4_7_Pasture_area_ha_per_lps']=0
    
    data={} # intermediate results data frame (for model variables not part of main results)
    
    data['sd_herd_size_TLU']=0
    
            

        

# Initialize simulations

    for ss in subsectors:      
        for b in breed:
            if (herd_size[(ss,b,'herd_pop')] ==0):
                continue
                
            for c in cohort:
                
                # initialize variables to evaluate seasonal (monthly) nutrient availability per animal (crude protein and metabolisable energy)
                                
                for i in range(0,12):
                    mo_me_short[ss,b,c,i]  = 0
                    mo_mp_short[ss,b,c,i]  = 0
                    mo_dp_cp[ss,b,c,i]  = 0
                    mo_dp_me[ss,b,c,i]  = 0
                    
                    mo_fi_pt[ss,b,c,i]=0
                    mo_fi_nhy[ss,b,c,i]=0
                    mo_fi_np[ss,b,c,i]=0
                    mo_fi_sto[ss,b,c,i]=0
                    mo_fi_mb[ss,b,c,i]=0
                    mo_fi_sc[ss,b,c,i]=0
                    mo_fi_grs[ss,b,c,i]=0
                    mo_fi_ns[ss,b,c,i]=0
                    

                # specify animal parameters for current run
            
                if (c == 'cow'):
                    if (b == 'local'):
                        bc_count=0
                        start_age=28
                        max_simu_range=12*13-20
                        if (scenario_parameters[(ss,'artificial_insemination',s)]==1):
                            d = Cow(age=start_age,breed='Zebu -- TZ south high')
                            d.sex='f'
                            d.parameters['potential_annual_calving_rate']['value']=1.0

                        else:
                            d = Cow(age=start_age,breed='Zebu -- TZ south high') 
                            d.parameters['potential_annual_calving_rate']['value']=0.45
                            d.sex='f'

                    else:
                        bc_count=5
                        start_age=20
                        max_simu_range=12*13-20
                        if (scenario_parameters[(ss,'artificial_insemination',s)]==1):
                            d = Cow(breed='Improved Tanz southern highlands')
                            d.sex='f'
                            d.parameters['potential_annual_calving_rate']['value']=1.0
                        else:
                            d = Cow(breed='Improved Tanz southern highlands') 
                            d.sex='f'
                            
                elif (c == 'bull'):
                    start_age=36
                    max_simu_range=24
                    if (b == 'local'):
                        bc_count=3
                        d=Cow(age=start_age,breed='Zebu -- TZ south high')
                        d.sex='m'
                    else:
                        bc_count=8
                        d=Cow(age=start_age,breed='Improved Tanz southern highlands')
                        d.sex='m'
                
                elif (c == 'ml_calf'):
                    start_age=1
                    max_simu_range=12
                    if (b == 'local'):
                        bc_count=2
                        d=Cow(age=start_age,breed='Zebu -- TZ south high')
                        d.sex='m'
                    else:
                        bc_count=7
                        d=Cow(age=start_age,breed='Improved Tanz southern highlands')
                        d.sex='m'

                elif (c == 'fe_calf'):
                    start_age=1
                    max_simu_range=12
                    if (b == 'local'):
                        bc_count=2
                        d=Cow(age=start_age,breed='Zebu -- TZ south high')
                        d.sex='f'
                    else:
                        bc_count=7
                        d=Cow(age=start_age,breed='Improved Tanz southern highlands')
                        d.sex='f'

                elif (c == 'heifer'):
                    start_age=12
                    max_simu_range=36
                    if (b == 'local'):
                        bc_count=1
                        d=Cow(age=start_age,breed='Zebu -- TZ south high')
                        d.sex='f'
                    else:
                        bc_count=6
                        d=Cow(age=start_age,breed='Improved Tanz southern highlands')
                        d.sex='f'
                
                elif (c == 'juv_male'):
                    start_age=12
                    max_simu_range=24
                    if (b == 'local'):
                        bc_count=4
                        d=Cow(age=start_age,breed='Zebu -- TZ south high')
                        d.sex='m'
                    else:
                        bc_count=9
                        d=Cow(age=start_age,breed='Improved Tanz southern highlands')
                        d.sex='m'
                      
                # general livsim parameters     
                d._main_variables = None 
                d._DETERMINISTIC= True
                d.is_bull_present = 1
                d._MAX_ITER = 20
                

                ############################### FEEDING SCRIPT ########################
                
                # define feeds used in simulations from library
                feeds=['grass','stover','napier','napier_hy','maize_bran','sunflower_seed_meal','pasture']

                GRASS=Feed(parameters='Native grass') 
                PASTURE=Feed(parameters='Pasture')
                NAPIER=Feed(parameters='napier grass') 
                NAPIER_HY=Feed(parameters='napier-legume') 
                NAPIER_SILAGE=Feed(parameters='napier silage') 
                SUNFLOWER_SEED_MEAL=Feed(parameters='sunflower seed meal')
                MAIZE_BRAN=Feed(parameters='maize bran')
                
                
                # Specify the diets for cows in line with the scenario
                feed_cow_base=diet_wb.sheet_by_index(0)
                
                if (b=='local'):
                    feed_cow_scen=diet_wb.sheet_by_index(int(scenario_parameters[(ss,'diet_id_cow_local',s)]))
   
                elif (b=='improved'):
                    feed_cow_scen=diet_wb.sheet_by_index(int(scenario_parameters[(ss,'diet_id_cow_improved',s)]))

                # Load stover nutrient properties 
                if ((c=='cow') and  (scenario_parameters[(ss,'urea_treatment',s)]==1)):
                    STOVER=Feed(parameters='maize stover urea treated')
                    m_stover="maize stover urea treated"
                    
                else:
                    STOVER=Feed(parameters='maize stover')
                    m_stover="maize stover"

                    
                # Define feed storage object
                feed_storage = FeedStorage()

                # Cow specific feed intake variables
                # Feed intake per day (kg dm/head/day) at specific stage of cow's production cycle (lactating, gestating, other)
                feed_intake_cow_early_lac={}
                feed_intake_cow_late_lac={}
                feed_intake_cow_gest={}
                feed_intake_cow_other={}
                feed_intake_cow_early_lac_base={}
                feed_intake_cow_late_lac_base={}
                feed_intake_cow_gest_base={}
                feed_intake_cow_other_base={}
                
                # Feed intake per year from specific production stages of cow (kg DM/head/yr)
                cons_per_year_early_lac_base={}
                cons_per_year_late_lac_base={}
                cons_per_year_lac_base={}
                cons_per_year_oth_base={}
                cons_per_year_early_lac={}
                cons_per_year_late_lac={}
                cons_per_year_lac={}
                cons_per_year_oth={}
                
                
                # All feed intake variables
                feed_intake={} # feed intake on an annual basis per breed/cohort (kg dm/yr/head)
                feed_intake_base={} # reference feed intake on an annual basis per breed/cohort (kg dm/yr/head)

                # Actual feed intake for this iteration
                # Feed for cows
                if (c =='cow'):
                    
                    lactation_duration=30*d.parameters['lactation_duration']['value']
                    early_lactation_duration=30*5
                    late_lactation_duration=lactation_duration-early_lactation_duration
                    calving_interval=365/d.parameters['potential_annual_calving_rate']['value']
                    FRAC_time_lactation=lactation_duration/calving_interval
                    FRAC_time_early_lactation=early_lactation_duration/calving_interval
                    FRAC_time_late_lactation=late_lactation_duration/calving_interval
                    days_not_in_lac=(1-FRAC_time_lactation)*365

                    # Load diets from excel
                    if (b=='local'):
                        breed_set=0
                    elif (b=='improved'):
                        breed_set=1
                        
                    for f in feeds:
                            row=(subsectors.index(ss)*3)+breed_set*(9)
                            feed_intake_cow_early_lac_base[ss,b,c,f]=feed_cow_base.cell(row+1,3+feeds.index(f)).value/1000
                            feed_intake_cow_late_lac_base[ss,b,c,f]=feed_cow_base.cell(row+2,3+feeds.index(f)).value/1000
                            feed_intake_cow_gest_base[ss,b,c,f]=feed_cow_base.cell(row+2,3+feeds.index(f)).value/1000
                            feed_intake_cow_other_base[ss,b,c,f]=feed_cow_base.cell(row+3,3+feeds.index(f)).value/1000
    
                    
                    cons_per_year_early_lac_base[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_early_lac_base[ss,b,c,'sunflower_seed_meal']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_late_lac_base[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_late_lac_base[ss,b,c,'sunflower_seed_meal']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_oth_base[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_other_base[ss,b,c,'sunflower_seed_meal']*days_not_in_lac

                    cons_per_year_early_lac_base[ss,b,c,'maize_bran']=feed_intake_cow_early_lac_base[ss,b,c,'maize_bran']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_late_lac_base[ss,b,c,'maize_bran']=feed_intake_cow_late_lac_base[ss,b,c,'maize_bran']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_oth_base[ss,b,c,'maize_bran']=feed_intake_cow_other_base[ss,b,c,'maize_bran']*days_not_in_lac

                    cons_per_year_early_lac_base[ss,b,c,'napier']=feed_intake_cow_early_lac_base[ss,b,c,'maize_bran']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_late_lac_base[ss,b,c,'napier']=feed_intake_cow_late_lac_base[ss,b,c,'maize_bran']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_oth_base[ss,b,c,'napier']=feed_intake_cow_other_base[ss,b,c,'maize_bran']*days_not_in_lac

                    cons_per_year_early_lac_base[ss,b,c,'napier_hy']=feed_intake_cow_early_lac_base[ss,b,c,'napier_hy']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_late_lac_base[ss,b,c,'napier_hy']=feed_intake_cow_late_lac_base[ss,b,c,'napier_hy']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_oth_base[ss,b,c,'napier_hy']=feed_intake_cow_other_base[ss,b,c,'napier_hy']*days_not_in_lac

                    cons_per_year_early_lac_base[ss,b,c,'grass']=feed_intake_cow_early_lac_base[ss,b,c,'maize_bran']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_late_lac_base[ss,b,c,'grass']=feed_intake_cow_late_lac_base[ss,b,c,'maize_bran']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_oth_base[ss,b,c,'grass']=feed_intake_cow_other_base[ss,b,c,'maize_bran']*days_not_in_lac

                    cons_per_year_early_lac_base[ss,b,c,'stover']=feed_intake_cow_early_lac_base[ss,b,c,'maize_bran']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_late_lac_base[ss,b,c,'stover']=feed_intake_cow_late_lac_base[ss,b,c,'maize_bran']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_oth_base[ss,b,c,'stover']=feed_intake_cow_other_base[ss,b,c,'maize_bran']*days_not_in_lac

                    cons_per_year_early_lac_base[ss,b,c,'pasture']=feed_intake_cow_early_lac_base[ss,b,c,'maize_bran']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_late_lac_base[ss,b,c,'pasture']=feed_intake_cow_late_lac_base[ss,b,c,'maize_bran']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_oth_base[ss,b,c,'pasture']=feed_intake_cow_other_base[ss,b,c,'maize_bran']*days_not_in_lac

                    # the annual feed intake of a feed category is equal to the sum over all production stages
                    feed_intake_base[ss,b,c,'napier']=cons_per_year_oth_base[ss,b,c,'napier']+cons_per_year_early_lac_base[ss,b,c,'napier']+cons_per_year_late_lac_base[ss,b,c,'napier']
                    feed_intake_base[ss,b,c,'napier_hy']=cons_per_year_oth_base[ss,b,c,'napier_hy']+cons_per_year_early_lac_base[ss,b,c,'napier_hy']+cons_per_year_late_lac_base[ss,b,c,'napier_hy']
                    feed_intake_base[ss,b,c,'maize_bran']=cons_per_year_oth_base[ss,b,c,'maize_bran']+cons_per_year_early_lac_base[ss,b,c,'maize_bran']+cons_per_year_late_lac_base[ss,b,c,'maize_bran']
                    feed_intake_base[ss,b,c,'sunflower_seed_meal']=cons_per_year_oth_base[ss,b,c,'sunflower_seed_meal']+cons_per_year_early_lac_base[ss,b,c,'sunflower_seed_meal']+cons_per_year_late_lac_base[ss,b,c,'sunflower_seed_meal']
                    feed_intake_base[ss,b,c,'grass']=cons_per_year_oth_base[ss,b,c,'grass']+cons_per_year_early_lac_base[ss,b,c,'grass']+cons_per_year_late_lac_base[ss,b,c,'grass']
                    feed_intake_base[ss,b,c,'stover']=cons_per_year_oth_base[ss,b,c,'stover']+cons_per_year_early_lac_base[ss,b,c,'stover']+cons_per_year_late_lac_base[ss,b,c,'stover']
                    feed_intake_base[ss,b,c,'pasture']=cons_per_year_oth_base[ss,b,c,'pasture']+cons_per_year_early_lac_base[ss,b,c,'pasture']+cons_per_year_late_lac_base[ss,b,c,'pasture']


                    # if the current scenario is a feeding scenario, diet is different from base
                    if (scenario_parameters[(ss,'diet_scenario',s)]==1):    
                        
                        # load diets from excel
                        for f in feeds:
                            row=(subsectors.index(ss)*3)+breed_set*9
                            
                            feed_intake_cow_early_lac[ss,b,c,f]=feed_cow_scen.cell(row+1,3+feeds.index(f)).value/1000
                            feed_intake_cow_late_lac[ss,b,c,f]=feed_cow_scen.cell(row+2,3+feeds.index(f)).value/1000
                            feed_intake_cow_gest[ss,b,c,f]=feed_cow_scen.cell(row+2,3+feeds.index(f)).value/1000
                            feed_intake_cow_other[ss,b,c,f]=feed_cow_scen.cell(row+3,3+feeds.index(f)).value/1000

                        
                        
                    else: ## if base scenario 

                        feed_intake_cow_early_lac[ss,b,c,'napier']=feed_intake_cow_early_lac_base[ss,b,c,'napier']
                        feed_intake_cow_late_lac[ss,b,c,'napier']=feed_intake_cow_late_lac_base[ss,b,c,'napier']
                        feed_intake_cow_gest[ss,b,c,'napier']=feed_intake_cow_gest_base[ss,b,c,'napier']
                        feed_intake_cow_other[ss,b,c,'napier']=feed_intake_cow_other_base[ss,b,c,'napier']
                            
                        feed_intake_cow_early_lac[ss,b,c,'napier_hy']=feed_intake_cow_early_lac_base[ss,b,c,'napier_hy']
                        feed_intake_cow_late_lac[ss,b,c,'napier_hy']=feed_intake_cow_late_lac_base[ss,b,c,'napier_hy']
                        feed_intake_cow_gest[ss,b,c,'napier_hy']=feed_intake_cow_gest_base[ss,b,c,'napier_hy']
                        feed_intake_cow_other[ss,b,c,'napier_hy']=feed_intake_cow_other_base[ss,b,c,'napier_hy']
  
                        feed_intake_cow_early_lac[ss,b,c,'maize_bran']=feed_intake_cow_early_lac_base[ss,b,c,'maize_bran']
                        feed_intake_cow_late_lac[ss,b,c,'maize_bran']=feed_intake_cow_late_lac_base[ss,b,c,'maize_bran']
                        feed_intake_cow_gest[ss,b,c,'maize_bran']=feed_intake_cow_gest_base[ss,b,c,'maize_bran']
                        feed_intake_cow_other[ss,b,c,'maize_bran']=feed_intake_cow_other_base[ss,b,c,'maize_bran']
                            
                        feed_intake_cow_early_lac[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_early_lac_base[ss,b,c,'sunflower_seed_meal']
                        feed_intake_cow_late_lac[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_late_lac_base[ss,b,c,'sunflower_seed_meal']
                        feed_intake_cow_gest[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_gest_base[ss,b,c,'sunflower_seed_meal']
                        feed_intake_cow_other[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_other_base[ss,b,c,'sunflower_seed_meal']
                        
                        feed_intake_cow_early_lac[ss,b,c,'grass']=feed_intake_cow_early_lac_base[ss,b,c,'grass']
                        feed_intake_cow_late_lac[ss,b,c,'grass']=feed_intake_cow_late_lac_base[ss,b,c,'grass']
                        feed_intake_cow_gest[ss,b,c,'grass']=feed_intake_cow_gest_base[ss,b,c,'grass']
                        feed_intake_cow_other[ss,b,c,'grass']=feed_intake_cow_other_base[ss,b,c,'grass']
                            
                        feed_intake_cow_early_lac[ss,b,c,'stover']=feed_intake_cow_early_lac_base[ss,b,c,'stover']
                        feed_intake_cow_late_lac[ss,b,c,'stover']=feed_intake_cow_late_lac_base[ss,b,c,'stover']
                        feed_intake_cow_gest[ss,b,c,'stover']=feed_intake_cow_gest_base[ss,b,c,'stover']
                        feed_intake_cow_other[ss,b,c,'stover']=feed_intake_cow_other_base[ss,b,c,'stover']
                        
                        feed_intake_cow_early_lac[ss,b,c,'pasture']=feed_intake_cow_early_lac_base[ss,b,c,'pasture']
                        feed_intake_cow_late_lac[ss,b,c,'pasture']=feed_intake_cow_late_lac_base[ss,b,c,'pasture']
                        feed_intake_cow_gest[ss,b,c,'pasture']=feed_intake_cow_gest_base[ss,b,c,'pasture']
                        feed_intake_cow_other[ss,b,c,'pasture']=feed_intake_cow_other_base[ss,b,c,'pasture']
                           
                    ## set annual consumption based on fraction of time in lactation
                        
                    cons_per_year_late_lac[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_late_lac[ss,b,c,'sunflower_seed_meal']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_early_lac[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_early_lac[ss,b,c,'sunflower_seed_meal']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_oth[ss,b,c,'sunflower_seed_meal']=feed_intake_cow_other[ss,b,c,'sunflower_seed_meal']*days_not_in_lac

                    cons_per_year_late_lac[ss,b,c,'maize_bran']=feed_intake_cow_late_lac[ss,b,c,'maize_bran']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_early_lac[ss,b,c,'maize_bran']=feed_intake_cow_early_lac[ss,b,c,'maize_bran']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_oth[ss,b,c,'maize_bran']=feed_intake_cow_other[ss,b,c,'maize_bran']*days_not_in_lac

                    cons_per_year_late_lac[ss,b,c,'napier']=feed_intake_cow_late_lac[ss,b,c,'napier']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_early_lac[ss,b,c,'napier']=feed_intake_cow_early_lac[ss,b,c,'napier']*lactation_duration*FRAC_time_early_lactation
                    cons_per_year_oth[ss,b,c,'napier']=feed_intake_cow_other[ss,b,c,'napier']*days_not_in_lac

                    cons_per_year_late_lac[ss,b,c,'napier_hy']=feed_intake_cow_late_lac[ss,b,c,'napier_hy']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_early_lac[ss,b,c,'napier_hy']=feed_intake_cow_early_lac[ss,b,c,'napier_hy']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_oth[ss,b,c,'napier_hy']=feed_intake_cow_other[ss,b,c,'napier_hy']*days_not_in_lac

                    cons_per_year_late_lac[ss,b,c,'grass']=feed_intake_cow_late_lac[ss,b,c,'grass']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_early_lac[ss,b,c,'grass']=feed_intake_cow_early_lac[ss,b,c,'grass']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_oth[ss,b,c,'grass']=feed_intake_cow_other[ss,b,c,'grass']*days_not_in_lac

                    cons_per_year_late_lac[ss,b,c,'stover']=feed_intake_cow_late_lac[ss,b,c,'stover']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_early_lac[ss,b,c,'stover']=feed_intake_cow_early_lac[ss,b,c,'stover']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_oth[ss,b,c,'stover']=feed_intake_cow_other[ss,b,c,'stover']*days_not_in_lac
                    
                    cons_per_year_late_lac[ss,b,c,'pasture']=feed_intake_cow_late_lac[ss,b,c,'pasture']*late_lactation_duration*FRAC_time_late_lactation
                    cons_per_year_early_lac[ss,b,c,'pasture']=feed_intake_cow_early_lac[ss,b,c,'pasture']*early_lactation_duration*FRAC_time_early_lactation
                    cons_per_year_oth[ss,b,c,'pasture']=feed_intake_cow_other[ss,b,c,'pasture']*days_not_in_lac
                    

                    ## sum intake over production stages to get the intake on an annual basis
                    feed_intake[ss,b,c,'napier']=cons_per_year_oth[ss,b,c,'napier']+cons_per_year_early_lac[ss,b,c,'napier']+cons_per_year_late_lac[ss,b,c,'napier']
                    feed_intake[ss,b,c,'napier_hy']=cons_per_year_oth[ss,b,c,'napier_hy']+cons_per_year_early_lac[ss,b,c,'napier_hy']+cons_per_year_late_lac[ss,b,c,'napier_hy']
                    feed_intake[ss,b,c,'maize_bran']=cons_per_year_oth[ss,b,c,'maize_bran']+cons_per_year_early_lac[ss,b,c,'maize_bran']+cons_per_year_late_lac[ss,b,c,'maize_bran']
                    feed_intake[ss,b,c,'sunflower_seed_meal']=cons_per_year_oth[ss,b,c,'sunflower_seed_meal']+cons_per_year_early_lac[ss,b,c,'sunflower_seed_meal']+cons_per_year_late_lac[ss,b,c,'sunflower_seed_meal']
                    feed_intake[ss,b,c,'grass']=cons_per_year_oth[ss,b,c,'grass']+cons_per_year_early_lac[ss,b,c,'grass']+cons_per_year_late_lac[ss,b,c,'grass']
                    feed_intake[ss,b,c,'stover']=cons_per_year_oth[ss,b,c,'stover']+cons_per_year_early_lac[ss,b,c,'stover']+cons_per_year_late_lac[ss,b,c,'stover']
                    feed_intake[ss,b,c,'pasture']=cons_per_year_oth[ss,b,c,'pasture']+cons_per_year_early_lac[ss,b,c,'pasture']+cons_per_year_late_lac[ss,b,c,'pasture']
  

                else: ## if not a cow, no stage specific feeding, and therefore annual intake is daily intake * 365
                    
                    feed_intake[ss,b,c,'grass']=365*sheet.cell_value(bc_count+1, 2)/1000
                    feed_intake[ss,b,c,'stover']=365*sheet.cell_value(bc_count+1, 3)/1000
                    feed_intake[ss,b,c,'napier']=365*sheet.cell_value(bc_count+1, 4)/1000 
                    feed_intake[ss,b,c,'napier_hy']=365*sheet.cell_value(bc_count+1, 5)/1000 
                    feed_intake[ss,b,c,'maize_bran']=365*sheet.cell_value(bc_count+1, 6)/1000 
                    feed_intake[ss,b,c,'sunflower_seed_meal']=365*sheet.cell_value(bc_count+1, 7)/1000 
                    feed_intake[ss,b,c,'pasture']=365*sheet.cell_value(bc_count+1, 8)/1000
            

                if (c != 'cow'):
                    feed_intake_base[ss,b,c,'grass']=365*sheet.cell_value(bc_count+1, 2)/1000
                    feed_intake_base[ss,b,c,'stover']=365*sheet.cell_value(bc_count+1, 3)/1000
                    feed_intake_base[ss,b,c,'napier']=365*sheet.cell_value(bc_count+1, 4)/1000
                    feed_intake_base[ss,b,c,'napier_hy']=365*sheet.cell_value(bc_count+1, 5)/1000 
                    feed_intake_base[ss,b,c,'maize_bran']=365*sheet.cell_value(bc_count+1, 6)/1000 
                    feed_intake_base[ss,b,c,'sunflower_seed_meal']=365*sheet.cell_value(bc_count+1, 7)/1000 
                    feed_intake_base[ss,b,c,'pasture']=365*sheet.cell_value(bc_count+1, 8)/1000 
                    
                    
                ## LAND FOOTPRINT ##
                # This section takes the feed intake for the current animal as an annual value and calculates area of land #
                # needed to produce this feed. This is done for both the reference (base) scenario and the current iteration,#
                # which is needed in order to calculate LUC emissions between scenarios #
                
                ## It is important to understand that in this framework, the concentrate feeds are fed according to the 
                # production cycles of the cow (lactating, gestating, etc.) whereas the other feeds are specifed as annual intakes,
                # and then the intake per month depends on the seasonality parameters (e.g the variation in yields of feeds, and rationing parameters, etc.)
                

                # reference yield (Mg DM/ha/yr) (corrected for efficiency of harvest/grazing)
                grass_annual_yield=(utn_efy[('grass')])*yield_sheet.cell_value(1, 2)
                pasture_annual_yield=(utn_efy[('pasture')])*yield_sheet.cell_value(2, 2)
                napier_annual_yield=(utn_efy[('napier')])*yield_sheet.cell_value(3, 2)
                napier_hy_annual_yield=(utn_efy[('napier_hy')])*yield_sheet.cell_value(4, 2)
                stover_annual_yield=(utn_efy[('stover')])*yield_sheet.cell_value(5, 2)
                maize_annual_yield=(utn_efy[('maize_bran')])*yield_sheet.cell_value(6, 2)*(scenario_parameters[(ss,'yield_scale_factor_maize',s)])
                sunflower_annual_yield=(utn_efy[('sunflower_seed_meal')])*yield_sheet.cell_value(7, 2)*(scenario_parameters[(ss,'yield_scale_factor_sunflower',s)])
                
                # reference area (ha)
                grass_area_base[ss,b,c]=(1/1000)*feed_intake_base[ss,b,c,'grass']/grass_annual_yield
                pasture_area_base[ss,b,c]=(1/1000)*feed_intake_base[ss,b,c,'pasture']/pasture_annual_yield
                stover_area_base[ss,b,c]=(1/1000)*(feed_intake_base[ss,b,c,'stover'])/stover_annual_yield
                sunflower_as_meal_area_base[ss,b,c]=(1/1000)*feed_intake_base[ss,b,c,'sunflower_seed_meal']/(scenario_parameters[(ss,'yield_scale_factor_sunflower','BAU')]*sunflower_annual_yield)
                maize_as_bran_area_base[ss,b,c]=(1/1000)*feed_intake_base[ss,b,c,'maize_bran']/(scenario_parameters[(ss,'yield_scale_factor_maize','BAU')]*maize_annual_yield)
                napier_area_base[ss,b,c]=(1/1000)*feed_intake_base[ss,b,c,'napier']/napier_annual_yield       
                napier_hy_area_base[ss,b,c]=(1/1000)*feed_intake_base[ss,b,c,'napier_hy']/napier_annual_yield       
                
                
                # summarize area for total feed, primary feeds, and primary crops
                feed_area_base[(ss,b,c)]=pasture_area_base[ss,b,c]+grass_area_base[ss,b,c]+stover_area_base[ss,b,c]+sunflower_as_meal_area_base[ss,b,c]+maize_as_bran_area_base[ss,b,c]+napier_area_base[ss,b,c]+napier_hy_area_base[ss,b,c]
                feed_primary_area_base[(ss,b,c)]= pasture_area_base[ss,b,c]+grass_area_base[ss,b,c]+sunflower_as_meal_area_base[ss,b,c]+maize_as_bran_area_base[ss,b,c]+napier_area_base[ss,b,c]+napier_hy_area_base[ss,b,c]
                crop_primary_area_base[(ss,b,c)]=sunflower_as_meal_area_base[ss,b,c]+maize_as_bran_area_base[ss,b,c]

                # actual area for this simulation (ha)
                pasture_area[ss,b,c]=(1/1000)*feed_intake[ss,b,c,'pasture']/(pasture_annual_yield)
                grass_area[ss,b,c]=(1/1000)*feed_intake[ss,b,c,'grass']/(grass_annual_yield)
                stover_area[ss,b,c]=(1/1000)*(feed_intake[ss,b,c,'stover'])/(stover_annual_yield)                
                sunflower_as_meal_area[ss,b,c]=(1/1000)*feed_intake[ss,b,c,'sunflower_seed_meal']/(sunflower_annual_yield*scenario_parameters[(ss,'yield_scale_factor_sunflower',s)])
                maize_as_bran_area[ss,b,c]=(1/1000)*feed_intake[ss,b,c,'maize_bran']/(maize_annual_yield*scenario_parameters[(ss,'yield_scale_factor_sunflower',s)])
                napier_area[ss,b,c]=(1/1000)*feed_intake[ss,b,c,'napier']/(napier_annual_yield)            
                napier_hy_area[ss,b,c]=(1/1000)*feed_intake[ss,b,c,'napier_hy']/(napier_hy_annual_yield) 
                
                # summarize area for total feed, primary feeds, and primary crops
                feed_area[(ss,b,c)]=grass_area[ss,b,c]+stover_area[ss,b,c]+sunflower_as_meal_area[ss,b,c]+maize_as_bran_area[ss,b,c]+napier_area[ss,b,c]+pasture_area[ss,b,c]
                feed_primary_area[(ss,b,c)]=grass_area[ss,b,c]+sunflower_as_meal_area[ss,b,c]+maize_as_bran_area[ss,b,c]+napier_area[ss,b,c]+pasture_area[ss,b,c]
                crop_primary_area[(ss,b,c)]=sunflower_as_meal_area[ss,b,c]+maize_as_bran_area[ss,b,c]
                total_grasslands_area[(ss,b,c)]=grass_area[ss,b,c]+napier_area[ss,b,c]+pasture_area[ss,b,c]+napier_hy_area[ss,b,c]
                #+napier_area[ss,b,c]
                
                # Define feed available for feeding based on yield and area (defined as 'production') (kg DM/animal/d for every month)
                
                pasture_production=[]## production available for feeding (kg DM/head/day)
                napier_production=[] ## production available for feeding (kg DM/head/day)
                napier_hy_production=[] ## production available for feeding (kg DM/head/day)
                napier_silage_production=[] ## production available for feeding (kg DM/head/day)
                grass_production=[]  ## production available for feeding (kg DM/head/day)
                stover_production=[] ## production available for feeding (kg DM/head/day)
                maize_bran_production=[] ## production available for feeding (kg DM/head/day)
                sunflower_seed_meal_production=[] ## production available for feeding (kg DM/head/day)
                
                
                for i in range(0,12):
                
                    napier_production.append(0)
                    napier_hy_production.append(0)
                    napier_silage_production.append(0)
                    grass_production.append(0)
                    stover_production.append(0)
                    pasture_production.append(0)
                
                
                # For maize bran and sunflower cake there is no seasonal variation and therefore 
                # monthly consumption is simply annual production divided by 12
                
                sunflower_yld=sunflower_annual_yield*1000/365
                maize_bran_yld=maize_annual_yield*1000/365

                maize_bran_production=maize_as_bran_area[ss,b,c]*maize_bran_yld
                sunflower_seed_meal_production=sunflower_as_meal_area[ss,b,c]*sunflower_yld

                # Define feed available in each month for stovers, pasture, and fodder crops, based on seasonality parameters
                # as defined in SM of article
                
                # Specify months pertaining to different seasons 
                # The first 6 months of the year are set as the dry season, the last 6 are the rainy season
                
            
                am=[0,1,2,3,4,5,6,7,8,9,10,11]  ## all months
                ld=[0,1,2,3,4,5]  # long dry season
                lr=[6,7,8,9,10,11] # long rainy season
                sr1=[0,1,2,3,4,5]  # months in which stover is rationed (after harvest) (see SM to article)
                
                
                # These two parameters are used to specify how grass/fodder yields vary by month
                # based on the seasonal yield fluctuations are described in article
                
                wsgn= 1.174 ## wet season gain  in yield of fodder and grass 
                dsdn= (12-len(ld)*(wsgn))/len(ld)  ## dry season drop in yield of fodder and grass 
                wsg_p= 1.48
                dsd_p=(12-len(ld)*(wsg_p))/len(ld)  ## dry season drop in yield of fodder and grass 
            
                # define the fraction of fodder from wet season that is ensiled for dry season feeding (only for cows)   
                
                ensilage_fodder_frac={}
                
                if (c == 'cow'):
                    ensilage_fodder_frac[(ss,b)]=scenario_parameters[(ss,'ensilage_fodder_frac',s)]
              
                elif (c!='cow'):
                    ensilage_fodder_frac[(ss,b)]=0
                   

                # For every month of the year specify feed available based on land footprint and seasonality parameters
                for i in lr:
                    napier_production[i]=wsgn*(1/12)*(1-ensilage_fodder_frac[(ss,b)])*napier_area[ss,b,c]*1000*napier_annual_yield/(30)
                    grass_production[i]=wsg_p*(1/30)*grass_area[ss,b,c]*1000*grass_annual_yield/12
                    pasture_production[i]=wsg_p*(1/30)*pasture_area[ss,b,c]*1000*pasture_annual_yield/12
                    napier_silage_production[i]=0
                    stover_production[i]=(.2)*(1/6)*stover_area[ss,b,c]*1000*stover_annual_yield/30
                
                for i in ld:
                    napier_production[i]=dsdn*(1/12)*napier_area[ss,b,c]*1000*napier_annual_yield/(30)
                    napier_hy_production[i]=dsdn*(1/12)*napier_hy_area[ss,b,c]*1000*napier_hy_annual_yield/(30)
                    grass_production[i]=(1-ensilage_fodder_frac[(ss,b)])*dsd_p*(1/30)*grass_area[ss,b,c]*1000*grass_annual_yield/12
                    pasture_production[i]=(1-ensilage_fodder_frac[(ss,b)])*dsd_p*(1/30)*pasture_area[ss,b,c]*1000*pasture_annual_yield/12
                    napier_silage_production[i]=(1/6)*(ensilage_fodder_frac[(ss,b)])*(wsgn*(1/2)*napier_area[ss,b,c]*1000*napier_annual_yield/(30))
                    
                # Stover is rationed equally over 6 months of the year (immediately following crop harvest)    
                for i in sr1:
                    stover_production[i]=(.8)*(1/6)*stover_area[ss,b,c]*1000*stover_annual_yield/30
                       
                ## Specify actual diets for the current animal based on the above simulated seasonal feed availability
                
                if (c =='cow'):
    
                    # Specify feed intakes for cows
                    # For concentrates feeds are specified based on production cycle
                    # For non-concentrates, feeds are specified by month based on seasonality of these feeds as defined above
                    
                    feed_storage.add_external_roughage(grass_production,'Native grass', 'default')
                    feed_storage.add_external_roughage(pasture_production ,'Pasture', 'default')
                    feed_storage.add_external_roughage(stover_production, m_stover, 'default')
                    feed_storage.add_external_roughage(napier_production, 'napier grass', 'default')
                    feed_storage.add_external_roughage(napier_silage_production, 'napier silage', 'default')
                    feed_storage.add_external_roughage(napier_hy_production, 'napier-legume', 'default')
                
                    feed_storage.add_external_concentrate(feed_intake_cow_early_lac[ss,b,c,'maize_bran'],'maize bran', 'early lactation')
                    feed_storage.add_external_concentrate(feed_intake_cow_early_lac[ss,b,c,'sunflower_seed_meal'],'sunflower seed meal', 'early lactation')

                    feed_storage.add_external_concentrate(feed_intake_cow_late_lac[ss,b,c,'maize_bran'],'maize bran', 'lactating')
                    feed_storage.add_external_concentrate(feed_intake_cow_late_lac[ss,b,c,'sunflower_seed_meal'],'sunflower seed meal', 'lactating')

                   
                    feed_storage.add_external_concentrate(feed_intake_cow_other[ss,b,c,'maize_bran'],'maize bran', 'default')
                    feed_storage.add_external_concentrate(feed_intake_cow_other[ss,b,c,'sunflower_seed_meal'],'sunflower seed meal', 'default')
                
                    
                else: 
                    feed_storage.add_external_roughage(pasture_production,'Pasture', 'default')
                    feed_storage.add_external_concentrate(maize_bran_production,'maize bran', 'default')
                    feed_storage.add_external_concentrate(sunflower_seed_meal_production,'sunflower seed meal', 'default')
                    feed_storage.add_external_roughage(napier_production, 'napier grass', 'default')
                    feed_storage.add_external_roughage(napier_hy_production, 'napier-legume', 'default')
                    feed_storage.add_external_roughage(napier_silage_production, 'napier silage', 'default')
                    feed_storage.add_external_roughage(grass_production, 'Native grass', 'default')
                    feed_storage.add_external_roughage(stover_production, 'maize stover', 'default')

                d.feed_supply=feed_storage

                #  RUN LIVSIM SIMULATIONS for a given breed and cohort #

                output_df = d.run(max_simu_range)
                
                final_month = start_age + len(output_df)

                #######

                
                ### Results and summation/scaling ###
                # Calculate the needed output parameters as averages from the entire simulation period.
                # This is performed by summing all output variables over simulation period, then dividing by time span
                # of simulation period to get annual averages
               
                start_lact = 0 # has lactation started for cow , boolean
                start_count = 0 # has the counting period for simulation started, boolean
                months_lactating=0 # number of months cow has been lactating
                simu_period=0 # months of simulation period
                cmc=0 # calendar month count. Used for keeping track of the season/month when calculating outputs from livsim
                
                # initialize calendar month variables as zero
                for i in range(0,12):
                    count_calendar_month[i,ss,b,c]=0
                
                

                # tallying variables
                total_milk_yield = 0 ## kg/cow
                faecal_Nitrogen_tally=0  # g N
                urinary_Nitrogen_tally=0 # g N
                dry_matter_intake_tally=0 # kg DM
                diet_Nitrogen_tally=0 # kg
                frac_conc_tally=0 # fraction
                frac_roughage_tally=0  # fraction
                neutral_detergent_fibre_tally=0 # kg 
                acid_detergent_fibre_tally=0 # kg
                metabolisable_energy_tally=0 # MJ
                feed_intake_grass_tally=0 #kg
                feed_intake_stover_tally=0 #kg
                feed_intake_napier_tally=0 #kg
                feed_intake_napier_hy_tally=0 #kg
                feed_intake_pasture_tally=0 #kg
                feed_intake_maize_bran_tally=0  #kg
                feed_intake_sfsm_tally=0  #kg
                crude_protein_tally=0  #kg

                # summation variables
                lifetime_milk = 0  # kg/cow
                total_faecal_N=0   # g N
                total_urinary_N=0  # g N
                total_gross_energy=0 # MJ 
                total_frac_rough=0 # fraction
                total_frac_conc=0 # fraction
                total_ME=0 # MJ
                total_Ym=0 # %
                total_DMI=0 # kg
                total_VS=0 # kg
                total_ADF=0 # kg
                total_NDF=0 # kg
                total_diet_digestibility=0 # fraction
                total_metabolisable_energy=0
                total_diet_Nitrogen=0 # kg
                total_crude_protein=0 # kg
                
                

                for m in range(len(output_df)):

                    # determine when to start counting (because female cattle only become cows after beginning of first lactation)
                    if ((((d.sex == 'f') and output_df.get_value(m,'Cow_milk_yield') > 0 ) and (start_lact == 0)) or ((d.sex == 'f') and (start_age == m))):
                                                                                                                 
                    if (c == 'cow'):
                            if ((output_df.get_value(m,'Cow_age')-start_age)<12):
                                cmc=(output_df.get_value(m,'Cow_age') - start_age)
                                cmc=0
                            elif ((output_df.get_value(m,'Cow_age')-start_age)>12):
                                cmc=int(12*(((output_df.get_value(m,'Cow_age') - start_age)/12)-1))
                                cmc=0
                        else:
                            cmc=0
                        start_count = m
                        
                    # If animal is a male start counting immediately
                    elif ((d.sex == 'm') and (start_count ==0)):
                        start_count = m
                        

                    if (start_count > 0 ):
                        simu_period+=1
                        
                        if (output_df.get_value(m,'Cow_dead') == 'True'):
                            print('Uh oh, ', b, ' ', c,' is dead')
                            break
                            
                            
                        # sum results variables over simulation period
                            
                        if (c == 'cow'):
                            lifetime_milk+=output_df.get_value(m,'Cow_milk_yield')

                            if (output_df.get_value(m,'Cow_is_lactating')):
                                
                                months_lactating+=1
                                
                                if (output_df.get_value(m,'Cow_months_after_calving')<=5):
                                    feed_intake_maize_bran=feed_intake_cow_early_lac[ss,b,c,'maize_bran']*MAIZE_BRAN.__getattribute__('DM')*(1/1000)*30                
                                    feed_intake_sfsm=feed_intake_cow_early_lac[ss,b,c,'sunflower_seed_meal']*SUNFLOWER_SEED_MEAL.__getattribute__('DM')*(1/1000)*30         
                                
                                elif (output_df.get_value(m,'Cow_months_after_calving')>5):
                                    feed_intake_maize_bran=feed_intake_cow_late_lac[ss,b,c,'maize_bran']*MAIZE_BRAN.__getattribute__('DM')*(1/1000)*30                
                                    feed_intake_sfsm=feed_intake_cow_late_lac[ss,b,c,'sunflower_seed_meal']*SUNFLOWER_SEED_MEAL.__getattribute__('DM')*(1/1000)*30         
                                
                                
                                feed_intake_grass=grass_production[cmc]*GRASS.__getattribute__('DM')*(1/1000)*30                  
                                feed_intake_pasture=pasture_production[cmc]*PASTURE.__getattribute__('DM')*(1/1000)*30                  
                                feed_intake_stover=stover_production[cmc]*STOVER.__getattribute__('DM')*(1/1000)*30                   
                                feed_intake_napier=napier_production[cmc]*NAPIER.__getattribute__('DM')*(1/1000)*30                  
                                feed_intake_napier_hy=napier_hy_production[cmc]*NAPIER_HY.__getattribute__('DM')*(1/1000)*30                  
                                feed_intake_napier_silage=napier_silage_production[cmc]*NAPIER_SILAGE.__getattribute__('DM')*(1/1000)*30                  

                        
                            else:
                                
                                feed_intake_sfsm=feed_intake_cow_other[ss,b,c,'sunflower_seed_meal']*SUNFLOWER_SEED_MEAL.__getattribute__('DM')*(1/1000)*30            
                                feed_intake_maize_bran=feed_intake_cow_other[ss,b,c,'maize_bran']*MAIZE_BRAN.__getattribute__('DM')*(1/1000)*30           
                                

                                feed_intake_grass=grass_production[cmc]*GRASS.__getattribute__('DM')*(1/1000)*30                  
                                feed_intake_pasture=pasture_production[cmc]*PASTURE.__getattribute__('DM')*(1/1000)*30                  
                                feed_intake_stover=stover_production[cmc]*STOVER.__getattribute__('DM')*(1/1000)*30 
                                feed_intake_napier=napier_production[cmc]*NAPIER.__getattribute__('DM')*(1/1000)*30 
                                feed_intake_napier_hy=napier_hy_production[cmc]*NAPIER_HY.__getattribute__('DM')*(1/1000)*30                  
                                feed_intake_napier_silage=napier_silage_production[cmc]*NAPIER_SILAGE.__getattribute__('DM')*(1/1000)*30                  
                        
                        
                        else:

                            feed_intake_maize_bran=maize_bran_production*MAIZE_BRAN.__getattribute__('DM')*(1/1000)*30               
                            feed_intake_sfsm=sunflower_seed_meal_production*SUNFLOWER_SEED_MEAL.__getattribute__('DM')*(1/1000)*30             
                            feed_intake_grass=grass_production[cmc]*GRASS.__getattribute__('DM')*(1/1000)*30                
                            feed_intake_stover=stover_production[cmc]*STOVER.__getattribute__('DM')*(1/1000)*30                  
                            feed_intake_napier=napier_production[cmc]*NAPIER.__getattribute__('DM')*(1/1000)*30 
                            feed_intake_napier_hy=napier_hy_production[cmc]*NAPIER_HY.__getattribute__('DM')*(1/1000)*30
                            feed_intake_pasture=pasture_production[cmc]*PASTURE.__getattribute__('DM')*(1/1000)*30                  

                        
                        total_rough_supply=feed_intake_pasture+feed_intake_napier+feed_intake_napier_hy+feed_intake_grass+feed_intake_stover+feed_intake_napier_silage
                        total_conc_supply=feed_intake_maize_bran+feed_intake_sfsm
                        
                        
                        if (total_conc_supply > 0):
                            fraction_concentrate_supply_eaten=output_df.get_value(m,'FI_concentrate_intake')/total_conc_supply
                        else:
                            fraction_concentrate_supply_eaten = 1
                            
                            
                        fraction_roughage_supply_eaten=output_df.get_value(m,'FI_roughage_intake')/total_rough_supply
                        faecal_Nitrogen_tally +=output_df.get_value(m,'GrPr_faecal_N')
                        urinary_Nitrogen_tally+=output_df.get_value(m,'GrPr_urinary_N')
                        frac_conc_tally+=fraction_concentrate_supply_eaten
                        frac_roughage_tally+= fraction_roughage_supply_eaten
                        dry_matter_intake_tally += 30*output_df.get_value(m,'FI_feed_intake') 
                        neutral_detergent_fibre_tally +=30*fraction_concentrate_supply_eaten*(sunflower_seed_meal_production*SUNFLOWER_SEED_MEAL.parameters['NDF']['value']+maize_bran_production*MAIZE_BRAN.parameters['NDF']['value'])+ 30*fraction_roughage_supply_eaten*(napier_silage_production[cmc]*NAPIER_SILAGE.parameters['NDF']['value']  +       pasture_production[cmc]*PASTURE.parameters['NDF']['value'][cmc]+grass_production[cmc]*GRASS.parameters['NDF']['value'][cmc]+stover_production[cmc]*STOVER.parameters['NDF']['value'][cmc]+napier_production[cmc]*NAPIER.parameters['NDF']['value'][cmc]+napier_hy_production[cmc]*NAPIER_HY.parameters['NDF']['value'][cmc])
                        acid_detergent_fibre_tally += 30*fraction_concentrate_supply_eaten*(sunflower_seed_meal_production*SUNFLOWER_SEED_MEAL.parameters['ADF']['value']+maize_bran_production*MAIZE_BRAN.parameters['ADF']['value'])+30*fraction_roughage_supply_eaten*(napier_silage_production[cmc]*NAPIER_SILAGE.parameters['ADF']['value']       + pasture_production[cmc]*PASTURE.parameters['ADF']['value'][cmc]+grass_production[cmc]*GRASS.parameters['ADF']['value'][cmc]+stover_production[cmc]*STOVER.parameters['ADF']['value'][cmc]+napier_production[cmc]*NAPIER.parameters['ADF']['value'][cmc]+napier_hy_production[cmc]*NAPIER_HY.parameters['ADF']['value'][cmc]) 
                        crude_protein_tally += 30*fraction_concentrate_supply_eaten*(sunflower_seed_meal_production*SUNFLOWER_SEED_MEAL.parameters['CP']['value']+maize_bran_production*MAIZE_BRAN.parameters['CP']['value'])+30*fraction_roughage_supply_eaten*( napier_silage_production[cmc]*NAPIER_SILAGE.parameters['CP']['value'] +        pasture_production[cmc]*PASTURE.parameters['CP']['value'][cmc]+grass_production[cmc]*GRASS.parameters['CP']['value'][cmc]+stover_production[cmc]*STOVER.parameters['CP']['value'][cmc]+napier_production[cmc]*NAPIER.parameters['CP']['value'][cmc]+napier_hy_production[cmc]*NAPIER_HY.parameters['CP']['value'][cmc]) 
                        metabolisable_energy_tally += 30*(fraction_concentrate_supply_eaten*(sunflower_seed_meal_production*SUNFLOWER_SEED_MEAL.parameters['ME']['value']+maize_bran_production*MAIZE_BRAN.parameters['ME']['value'])+fraction_roughage_supply_eaten*( napier_silage_production[cmc]*NAPIER_SILAGE.parameters['ME']['value'][cmc]   +        pasture_production[cmc]*PASTURE.parameters['ME']['value'][cmc]+grass_production[cmc]*GRASS.parameters['ME']['value'][cmc]+stover_production[cmc]*STOVER.parameters['ME']['value'][cmc]+napier_production[cmc]*NAPIER.parameters['ME']['value'][cmc]+napier_hy_production[cmc]*NAPIER_HY.parameters['ME']['value'][cmc]))

                        # Cow_MP_shortage
                        mo_mp_short[ss,b,c,cmc] += output_df.get_value(m,'Cow_MP_shortage')
                        mo_me_short[ss,b,c,cmc] += output_df.get_value(m,'Cow_ME_shortage')
                        
                        mo_fi_mb[ss,b,c,cmc] += fraction_concentrate_supply_eaten*maize_bran_production
                        mo_fi_sc[ss,b,c,cmc] += fraction_concentrate_supply_eaten*sunflower_seed_meal_production
                        mo_fi_sto[ss,b,c,cmc] += fraction_roughage_supply_eaten*stover_production[cmc]
                        
                        mo_fi_pt[ss,b,c,cmc] += fraction_roughage_supply_eaten*pasture_production[cmc]
                        mo_fi_np[ss,b,c,cmc] += fraction_roughage_supply_eaten*napier_production[cmc]
                        mo_fi_nhy[ss,b,c,cmc] += fraction_roughage_supply_eaten*stover_production[cmc]
                        mo_fi_ns[ss,b,c,cmc] += fraction_roughage_supply_eaten*napier_silage_production[cmc]
                        mo_fi_grs[ss,b,c,cmc] += fraction_roughage_supply_eaten*grass_production[cmc]
                        
                        count_calendar_month[cmc,ss,b,c]+=1

                        feed_intake_grass_tally += fraction_roughage_supply_eaten*feed_intake_grass*GRASS.__getattribute__('DM')/1000              
                        feed_intake_stover_tally += fraction_roughage_supply_eaten*feed_intake_stover*STOVER.__getattribute__('DM')/1000                 
                        feed_intake_napier_tally += fraction_roughage_supply_eaten*(feed_intake_napier+feed_intake_napier_silage)*NAPIER.__getattribute__('DM')/1000                 
                        feed_intake_napier_hy_tally += fraction_roughage_supply_eaten*(feed_intake_napier_hy)*NAPIER.__getattribute__('DM')/1000                 
                        feed_intake_maize_bran_tally += fraction_concentrate_supply_eaten*feed_intake_maize_bran*MAIZE_BRAN.__getattribute__('DM')/1000         
                        feed_intake_sfsm_tally += fraction_concentrate_supply_eaten*feed_intake_sfsm*SUNFLOWER_SEED_MEAL.__getattribute__('DM')/1000                 
                        feed_intake_pasture_tally += fraction_roughage_supply_eaten*feed_intake_pasture*PASTURE.__getattribute__('DM')/1000

                        
                    if (cmc < 11):
                        cmc+=1
                    else: 
                        cmc=0
                        
                    
                    


                # Now calculate the average dietary and excretion parameters over the entire simulation period of the animal
                # These values then get stored in the results dataframe below

                total_faecal_N = 12*faecal_Nitrogen_tally/simu_period
                total_urinary_N = 12*urinary_Nitrogen_tally/simu_period
                total_NDF = 12*neutral_detergent_fibre_tally/simu_period
                total_ADF = 12*acid_detergent_fibre_tally/simu_period
                total_ME = 12*metabolisable_energy_tally/simu_period
                total_DMI = 12*dry_matter_intake_tally/simu_period
                total_gross_energy= 18.4*total_DMI
                total_crude_protein=12*crude_protein_tally/simu_period

                total_diet_Nitrogen = 12*(1/1000)*(1/6.25)*total_crude_protein/simu_period
                
                ## calculated variables
                
                total_diet_digestibility=83.58 - 0.824*100*(1/1000)*total_ADF/total_DMI+ 2.626*100*(1/1000)*(total_diet_Nitrogen)/total_DMI
                volatile_solids_per_cow_per_year=((18.4*total_DMI *total_diet_digestibility/100)+0.04*total_gross_energy)*(0.92/18.45)
                Ym_per_cow_per_year= 3.1 - 0.243*total_DMI/365 + 0.0059*total_NDF/total_DMI + 0.0057*10*total_diet_digestibility/100

                pasture_intake[ss,b,c]=12*feed_intake_pasture_tally/simu_period
                grass_intake[ss,b,c]=12*feed_intake_grass_tally/simu_period
                stover_intake[ss,b,c]=12*feed_intake_stover_tally/simu_period
                napier_intake[ss,b,c]=12*feed_intake_napier_tally/simu_period
                napier_hy_intake[ss,b,c]=12*feed_intake_napier_hy_tally/simu_period
                maize_bran_intake[ss,b,c]=12*feed_intake_maize_bran_tally/simu_period
                sfsm_intake[ss,b,c]=12*feed_intake_sfsm_tally/simu_period

                if (c == 'cow'):
                    ## convert milk to FPCM
                    milk_yield_reg[ss,b]=12*lifetime_milk*(0.337+.116*(1/10)*d.parameters['milk_butterfat_content']['value']+0.06*(1/10)*d.parameters['milk_crude_protein_content']['value'])/simu_period
                    milk_yield_lact[ss,b]=d.parameters['lactation_duration']['value']*lifetime_milk* (0.337+.116*(1/10)*d.parameters['milk_butterfat_content']['value']+0.06*(1/10)*d.parameters['milk_crude_protein_content']['value'])/months_lactating
                   
                    RESULTS['v3_0_simu_period_months_'+str(ss)+'_'+str(b)]=simu_period
                    RESULTS['v3_0_lactation_months_'+str(ss)+'_'+str(b)]=months_lactating
                    
                fraction_roughage_eaten[ss,b,c]=frac_roughage_tally/simu_period
                fraction_conc_eaten[ss,b,c]=frac_conc_tally/simu_period

                faecal_Nitrogen[ss,b,c]=total_faecal_N 
                urinary_Nitrogen[ss,b,c]=total_urinary_N               
                Ym[ss,b,c]=Ym_per_cow_per_year
                VS[ss,b,c]=volatile_solids_per_cow_per_year
                DMI[ss,b,c]=total_DMI
                DMD[ss,b,c]=total_diet_digestibility
                ME[ss,b,c]=total_ME /(simu_period)
                NDF[ss,b,c]=12*neutral_detergent_fibre_tally/(simu_period)
                ADF[ss,b,c]=12*acid_detergent_fibre_tally/(simu_period)
                crude_protein_intake[ss,b,c]=total_crude_protein/1000


                # agricultural soils N2O based on IPCC framework
                N_MMT_Frac={} # Fraction of Nitrogen excretion that is managed vs. excreted on pasture
                
                # for EFs specific to a breed, specify current iteration's EF
                if (b=='improved'):
                    MCF=MCF_IMP
                    FRAC_MANURE_PRP=FRAC_MANURE_PRP_IMP
                    FRAC_MANURE_MMT=FRAC_MANURE_MMT_IMP
                    EF_4=EF_4_IMP
                    EF_5=EF_5_IMP
                    FRAC_vol=FRAC_vol_IMP
                    FRAC_leach=FRAC_leach_IMP
                     
                elif (b=='local'):
                    MCF=MCF_LOC
                    FRAC_MANURE_PRP=FRAC_MANURE_PRP_LOC
                    FRAC_MANURE_MMT=FRAC_MANURE_MMT_LOC
                    EF_4=EF_4_LOC
                    EF_5=EF_5_LOC
                    FRAC_vol=FRAC_vol_LOC
                    FRAC_leach=FRAC_leach_LOC
                    
                
                N_manure_to_Pasture=(1/1000)*(faecal_Nitrogen[ss,b,c]+urinary_Nitrogen[ss,b,c])*FRAC_MANURE_PRP
                N_manure_to_Storage=(1/1000)*(faecal_Nitrogen[ss,b,c]*(1-EF_3_FAECAL)+urinary_Nitrogen[ss,b,c]*(1-EF_3_URINARY))*FRAC_MANURE_MMT
                
                N_manure_for_Pasture=N_manure_to_Pasture
                N_manure_for_Soil=N_manure_to_Storage*(1-FRAC_loss_MMS)
                
                # Allocate total n from manure management thats applied to soils to different land categories
                # This is a management parameter; how N from management is allocated to different land uses
                N_MMT_Frac['grass']=N_manure_for_Soil*grass_area[ss,b,c]/feed_area[(ss,b,c)]
                N_MMT_Frac['napier']=N_manure_for_Soil*napier_area[ss,b,c]/feed_area[(ss,b,c)]
                N_MMT_Frac['napier_hy']=N_manure_for_Soil*napier_hy_area[ss,b,c]/feed_area[(ss,b,c)]
                N_MMT_Frac['maize_bran']=N_manure_for_Soil*maize_as_bran_area[ss,b,c]/feed_area[(ss,b,c)]
                N_MMT_Frac['sunflower_seed_meal']=N_manure_for_Soil*sunflower_as_meal_area[ss,b,c]/feed_area[(ss,b,c)]
                N_MMT_Frac['stover']=N_manure_for_Soil*stover_area[ss,b,c]/feed_area[(ss,b,c)]
                
                # Fertilizer, manure, and cr residue inputs into land categories (kg n/ha/yr)
                Fsn={}
                Fon={}
                Fcr={}
                Fprp={}
                
                # This section accounts for all N inputs into different land categories based on IPCC chapter on Soils
                # This includes synthetic fertilizer (sn)
                # Organic fertilizer (fon) (manure)
                # Crop residues (fcr)
                # Manure deposited from grazing cattle
                
                
                # Synthetic fertilizer n (kg n/ha/yr)
                Fsn['grass']=0
                Fsn['pasture']=0
                Fsn['napier']=0
                Fsn['napier_hy']=0
                Fsn['maize_bran']=20+scenario_parameters[(ss,'nitrous_oxide_factor_maize',s)]
                Fsn['sunflower_seed_meal']=20+scenario_parameters[(ss,'nitrous_oxide_factor_sunflower',s)]
                Fsn['stover']=10
                
                # Organic fertilizer n (kg n/ha/yr)
                Fon['grass']=N_MMT_Frac['grass']/grass_area[ss,b,c]
                Fon['pasture']=0
                Fon['napier']=(N_MMT_Frac['napier']+N_MMT_Frac['maize_bran'])/napier_area[ss,b,c]
                Fon['napier_hy']=(N_MMT_Frac['napier_hy'])/napier_hy_area[ss,b,c] 
                Fon['maize_bran']=0 
                Fon['sunflower_seed_meal']=0
                Fon['stover']=(N_MMT_Frac['stover']+N_MMT_Frac['sunflower_seed_meal'])/stover_area[ss,b,c]
                
                # Residue n (kg n/ha/yr) (these are calculated outside of model based on IPCC 2006 equations)
                Fcr['grass']=2.2
                Fcr['pasture']=4.5
                Fcr['napier']=16.9
                Fcr['napier_hy']=16.9
                Fcr['maize_bran']=15.2
                Fcr['sunflower_seed_meal']=10.6
                Fcr['stover']=15.2
                
                # Tally total synthetic N applied to get emissions from manufacturing below
                Synth_N_tot_kg_yr=stover_area[ss,b,c]*Fsn['stover']+sunflower_as_meal_area[ss,b,c]*Fsn['sunflower_seed_meal']+Fsn['maize_bran']*maize_as_bran_area[ss,b,c]
                
                if (grass_area[ss,b,c]>0):
                    
                    Fprp['grass']=N_manure_for_Pasture/grass_area[ss,b,c]
                    
                elif (grass_area[ss,b,c]==0):
                    
                    Fprp['grass']=0
    
                # Direct N2O from all sources per land category (kg n2o-n/yr)
                soil_N2O_feed_direct['pasture']=pasture_area[ss,b,c]*(EF_1*(Fsn['pasture']+Fcr['pasture']+Fon['pasture']))
                soil_N2O_feed_direct['grass']=grass_area[ss,b,c]*(EF_1*(Fsn['grass']+Fcr['grass']+Fon['grass'])+EF_3_PRP*Fprp['grass'])
                soil_N2O_feed_direct['napier']=napier_area[ss,b,c]*EF_1*(Fsn['napier']+ Fon['napier']+Fcr['napier'])
                soil_N2O_feed_direct['napier_hy']=napier_area[ss,b,c]*EF_1*(Fsn['napier_hy']+ Fon['napier_hy']+Fcr['napier_hy'])
                soil_N2O_feed_direct['stover']=stover_area[ss,b,c]*allocation_factor_stover*EF_1*(Fsn['stover']+ Fon['stover']+Fcr['stover'])
                soil_N2O_feed_direct['maize_bran']=maize_as_bran_area[ss,b,c]*allocation_factor_maize_bran*EF_1*(Fsn['maize_bran']+Fcr['maize_bran'])
                soil_N2O_feed_direct['sunflower_seed_meal']=sunflower_as_meal_area[ss,b,c]*allocation_factor_sunflower_seed_meal*EF_1*(Fsn['sunflower_seed_meal']+Fcr['sunflower_seed_meal'])
                
                # indirect (volatilization) (kg n2o-n/yr)
                soil_N2O_feed_adn['pasture']=pasture_area[ss,b,c]*EF_4_adn*(Fsn['pasture']*FRAC_gasf+(Fon['pasture'])*FRAC_gasm)#
                soil_N2O_feed_adn['grass']=grass_area[ss,b,c]*EF_4_adn*(Fsn['grass']*FRAC_gasf+(Fon['grass']+Fprp['grass'])*FRAC_gasm)
                soil_N2O_feed_adn['napier']=napier_area[ss,b,c]*EF_4_adn*(Fsn['napier']*FRAC_gasf+Fon['napier']*FRAC_gasm)
                soil_N2O_feed_adn['napier_hy']=napier_area[ss,b,c]*EF_4_adn*(Fsn['napier_hy']*FRAC_gasf+Fon['napier_hy']*FRAC_gasm)
                soil_N2O_feed_adn['stover']=stover_area[ss,b,c]*allocation_factor_stover*EF_4_adn*(Fsn['stover']*FRAC_gasf+Fon['stover']*FRAC_gasm)
                soil_N2O_feed_adn['maize_bran']=maize_as_bran_area[ss,b,c]*allocation_factor_maize_bran*EF_4_adn*(Fsn['maize_bran']*FRAC_gasf+Fon['maize_bran']*FRAC_gasm)
                soil_N2O_feed_adn['sunflower_seed_meal']=sunflower_as_meal_area[ss,b,c]*allocation_factor_sunflower_seed_meal*EF_4_adn*(Fsn['sunflower_seed_meal']*FRAC_gasf+Fon['sunflower_seed_meal']*FRAC_gasm)
              
                # indirect (leaching) (kg n2o-n/yr)
                soil_N2O_feed_lch['pasture']=pasture_area[ss,b,c]*EF_5_lch*(Fsn['pasture']+Fcr['pasture']+Fon['pasture'])
                soil_N2O_feed_lch['grass']=grass_area[ss,b,c]*EF_5_lch*(Fsn['grass']+Fcr['grass']+Fon['grass']+Fprp['grass'])
                soil_N2O_feed_lch['napier']=napier_area[ss,b,c]*EF_5_lch*(Fsn['napier']+ Fon['napier']+Fcr['napier'])
                soil_N2O_feed_lch['napier_hy']=napier_area[ss,b,c]*EF_5_lch*(Fsn['napier_hy']+ Fon['napier_hy']+Fcr['napier_hy'])
                soil_N2O_feed_lch['stover']=stover_area[ss,b,c]*allocation_factor_stover*EF_5_lch*(Fsn['stover']+ Fon['stover']+Fcr['stover'])
                soil_N2O_feed_lch['maize_bran']=maize_as_bran_area[ss,b,c]*allocation_factor_maize_bran*EF_5_lch*(Fsn['maize_bran']+Fcr['maize_bran'])
                soil_N2O_feed_lch['sunflower_seed_meal']=sunflower_as_meal_area[ss,b,c]*allocation_factor_sunflower_seed_meal*EF_5_lch*(Fsn['sunflower_seed_meal']+Fcr['sunflower_seed_meal'])
               
                # total 
                soil_N2O_feed['pasture']=soil_N2O_feed_direct['pasture']+soil_N2O_feed_adn['pasture']+soil_N2O_feed_lch['pasture']
                soil_N2O_feed['grass']=soil_N2O_feed_direct['grass']+soil_N2O_feed_adn['grass']+soil_N2O_feed_lch['grass']
                soil_N2O_feed['napier']=soil_N2O_feed_direct['napier']+soil_N2O_feed_adn['napier']+soil_N2O_feed_lch['napier']
                soil_N2O_feed['napier_hy']=soil_N2O_feed_direct['napier_hy']+soil_N2O_feed_adn['napier_hy']+soil_N2O_feed_lch['napier_hy']
                soil_N2O_feed['stover']=soil_N2O_feed_direct['stover']+soil_N2O_feed_adn['stover']+soil_N2O_feed_lch['stover']
                soil_N2O_feed['maize_bran']=soil_N2O_feed_direct['maize_bran']+soil_N2O_feed_adn['maize_bran']+soil_N2O_feed_lch['maize_bran']
                soil_N2O_feed['sunflower_seed_meal']=soil_N2O_feed_direct['sunflower_seed_meal']+soil_N2O_feed_adn['sunflower_seed_meal']+soil_N2O_feed_lch['sunflower_seed_meal']
                
                
                # calculate GHG emissions from all sources
                gross_energy[ss,b,c]=18.4*DMI[ss,b,c]
                
                enteric_CH4[ss,b,c]=28*gross_energy[ss,b,c]*Ym[ss,b,c]/(100*55.65)
                
                manure_CH4[ss,b,c]=(28*VS[ss,b,c]*B0*MCF*0.67)
                manure_N2O_direct[ss,b,c]=265*(44/28)*FRAC_MANURE_MMT*((1/1000)*faecal_Nitrogen[ss,b,c]*EF_3_FAECAL+(1/1000)*urinary_Nitrogen[ss,b,c]*EF_3_URINARY)
                manure_N2O_vol[ss,b,c]=265*(44/28)*((1/1000)*FRAC_MANURE_MMT*(faecal_Nitrogen[ss,b,c]+urinary_Nitrogen[ss,b,c]))*FRAC_vol*EF_4
                manure_N2O_leach[ss,b,c]=265*(44/28)*((1/1000)*FRAC_MANURE_MMT*(faecal_Nitrogen[ss,b,c]+urinary_Nitrogen[ss,b,c]))*FRAC_leach*EF_5
                manure_N2O[ss,b,c]=manure_N2O_direct[ss,b,c]+manure_N2O_vol[ss,b,c]+manure_N2O_leach[ss,b,c]
               
                soil_N2O_direct_grassland[ss,b,c]=265*(44/28)*(soil_N2O_feed_direct['grass']+soil_N2O_feed_direct['napier_hy']+soil_N2O_feed_direct['napier']+soil_N2O_feed_direct['pasture'])
                soil_N2O_indirect_leach_grassland[ss,b,c]=265*(44/28)*(soil_N2O_feed_lch['grass']+soil_N2O_feed_lch['napier_hy']+soil_N2O_feed_lch['napier']+soil_N2O_feed_lch['pasture'])
                soil_N2O_indirect_vol_grassland[ss,b,c]=265*(44/28)*(soil_N2O_feed_adn['grass']+soil_N2O_feed_adn['napier_hy']+soil_N2O_feed_adn['napier']+soil_N2O_feed_adn['pasture'])
                
                soil_N2O_direct_cropland[ss,b,c]=265*(44/28)*(soil_N2O_feed_direct['maize_bran']+soil_N2O_feed_direct['sunflower_seed_meal']+soil_N2O_feed_direct['stover'])
                soil_N2O_indirect_leach_cropland[ss,b,c]=265*(44/28)*(soil_N2O_feed_lch['maize_bran']+soil_N2O_feed_lch['sunflower_seed_meal']+soil_N2O_feed_lch['stover'])
                soil_N2O_indirect_vol_cropland[ss,b,c]=265*(44/28)*(soil_N2O_feed_adn['maize_bran']+soil_N2O_feed_adn['sunflower_seed_meal']+soil_N2O_feed_adn['stover'])
                
                soil_N2O_indirect_cropland[ss,b,c]=soil_N2O_indirect_vol_cropland[ss,b,c]+soil_N2O_indirect_leach_cropland[ss,b,c]
                soil_N2O_cropland[ss,b,c]=soil_N2O_direct_cropland[ss,b,c]+soil_N2O_indirect_cropland[ss,b,c]
                
                soil_N2O_indirect_grassland[ss,b,c]=soil_N2O_indirect_vol_grassland[ss,b,c]+soil_N2O_indirect_leach_grassland[ss,b,c]
                soil_N2O_grassland[ss,b,c]=soil_N2O_direct_grassland[ss,b,c]+soil_N2O_indirect_grassland[ss,b,c]
                
                energy_use_CO2[ss,b,c]=feed_embodied_CO2['maize_bran']*maize_bran_production*365/1000 + feed_embodied_CO2['sunflower_seed_meal']*sunflower_seed_meal_production*365/1000
                energy_use_CO2[ss,b,c]= energy_use_CO2[ss,b,c]+Synth_N_tot_kg_yr*fertilizer_embodied_CO2
            
                # Allocate to milk
                # Allocation factor is 0.9
                alloc_factor=0.9
                enteric_CH4[ss,b,c]=alloc_factor*enteric_CH4[ss,b,c]
                manure_CH4[ss,b,c]=alloc_factor*manure_CH4[ss,b,c]
                manure_N2O_direct[ss,b,c]=alloc_factor*manure_N2O_direct[ss,b,c]
                manure_N2O_vol[ss,b,c]=alloc_factor*manure_N2O_vol[ss,b,c]
                manure_N2O_leach[ss,b,c]=alloc_factor*manure_N2O_leach[ss,b,c]
                manure_N2O[ss,b,c]=alloc_factor*manure_N2O[ss,b,c]
                soil_N2O_grassland[ss,b,c]=alloc_factor*soil_N2O_grassland[ss,b,c]
                soil_N2O_cropland[ss,b,c]=alloc_factor*soil_N2O_cropland[ss,b,c]
                energy_use_CO2[ss,b,c]=alloc_factor*energy_use_CO2[ss,b,c]


                # Monte Carlo simulations 
                
                uncertainty_sheet= data_wb.sheet_by_index(2)
                rvars=0
                random_var={}
                num_vars=uncertainty_sheet.nrows

                for r in range(0,MC_sims):
                    for i in range(0,num_vars+1):
                        random_var[r,i]=np.random.normal(100,uncertainty_sheet.cell_value(rvars+1, 1))/100

  
                    Ym_rd[ss,b,c]=Ym[ss,b,c]*random_var[r,0]
                    B0_rd[ss,b,c]=B0*random_var[r,1]
                    MCF_rd[ss,b,c]=MCF*random_var[r,2]
                    EF_3_FAECAL_rd[ss,b,c]=EF_3_FAECAL*random_var[r,3]
                    EF_3_URINARY_rd[ss,b,c]=EF_3_URINARY*random_var[r,4]
                    EF_4_rd=EF_4*random_var[r,5]
                    EF_5_rd=EF_5*random_var[r,6]
                    FRAC_vol_rd= FRAC_vol*random_var[r,7]
                    FRAC_leach_rd=FRAC_leach*random_var[r,8]
                    EF_1_rd=EF_1*random_var[r,9]
                    EF_3_PRP_rd=EF_3_PRP*random_var[r,10]
                    EF_4_adn_rd=EF_4_adn*random_var[r,11]
                    EF_5_lch_rd=EF_5_lch*random_var[r,12]
                    

                    # Pasture
                    if (s == 'Base'):
                        grass_area_rd=grass_area[ss,b,c]*random_var[r,14]
                        pasture_area_rd=pasture_area[ss,b,c]*random_var[r,14]
                        napier_area_rd=napier_area[ss,b,c]*random_var[r,15]
                        napier_hy_area_rd=napier_area[ss,b,c]*random_var[r,15]
                        maize_as_bran_area_rd=maize_as_bran_area[ss,b,c]*random_var[r,16]
                        sunflower_as_meal_area_rd=sunflower_as_meal_area[ss,b,c]*random_var[r,17]
                        stover_area_rd=stover_area[ss,b,c]*random_var[r,18]
                        herd_size_rd[(ss,b,'herd_pop')]=herd_size[(ss,b,'herd_pop')]*random_var[r,19]
                        
                    else:
                        grass_area_rd=grass_area[ss,b,c]
                        pasture_area_rd=pasture_area[ss,b,c]
                        napier_area_rd=napier_area[ss,b,c]
                        napier_hy_area_rd=napier_area[ss,b,c]
                        maize_as_bran_area_rd=maize_as_bran_area[ss,b,c]
                        sunflower_as_meal_area_rd=sunflower_as_meal_area[ss,b,c]
                        stover_area_rd=stover_area[ss,b,c]
                        herd_size_rd[(ss,b,'herd_pop')]=herd_size[(ss,b,'herd_pop')]
                        
                    crop_primary_area_rd=crop_primary_area[ss,b,c]*random_var[r,23]
                    feed_primary_area_rd=feed_primary_area[ss,b,c]*random_var[r,24]
                        
                    
                # direct n2o from all sources per land category (kg n2o-n/yr)
                    soil_N2O_feed_direct_rd['pasture']=pasture_area_rd*(EF_1_rd*(Fsn['pasture']+Fcr['pasture']))
                    soil_N2O_feed_direct_rd['grass']=grass_area_rd*(EF_1_rd*(Fsn['grass']+Fcr['grass']+Fon['grass'])+EF_3_PRP_rd*Fprp['grass'])
                    soil_N2O_feed_direct_rd['napier']=napier_area_rd*EF_1_rd*(Fsn['napier']+ Fon['napier']+Fcr['napier'])
                    soil_N2O_feed_direct_rd['napier_hy']=napier_hy_area_rd*EF_1_rd*(Fsn['napier_hy']+ Fon['napier_hy']+Fcr['napier_hy'])
                    soil_N2O_feed_direct_rd['stover']=stover_area_rd*allocation_factor_stover*EF_1_rd*(Fsn['stover']+ Fon['stover']+Fcr['stover'])
                    soil_N2O_feed_direct_rd['maize_bran']=maize_as_bran_area_rd*allocation_factor_maize_bran*EF_1_rd*(Fsn['maize_bran']+Fcr['maize_bran'])
                    soil_N2O_feed_direct_rd['sunflower_seed_meal']=sunflower_as_meal_area_rd*allocation_factor_sunflower_seed_meal*EF_1_rd*(Fsn['sunflower_seed_meal']+Fcr['sunflower_seed_meal'])
                
                # indirect (volatilization) (kg n2o-n/yr)
                    soil_N2O_feed_adn_rd['pasture']=pasture_area_rd*EF_4_adn_rd*(Fsn['pasture']*FRAC_gasf+(Fon['pasture'])*FRAC_gasm)
                    soil_N2O_feed_adn_rd['grass']=grass_area_rd*EF_4_adn_rd*(Fsn['grass']*FRAC_gasf+(Fon['grass']+Fprp['grass'])*FRAC_gasm)
                    soil_N2O_feed_adn_rd['napier']=napier_area_rd*EF_4_adn_rd*(Fsn['napier']*FRAC_gasf+Fon['napier']*FRAC_gasm)
                    soil_N2O_feed_adn_rd['napier_hy']=napier_hy_area_rd*EF_4_adn_rd*(Fsn['napier_hy']*FRAC_gasf+Fon['napier_hy']*FRAC_gasm)
                    soil_N2O_feed_adn_rd['stover']=stover_area_rd*allocation_factor_stover*EF_4_adn_rd*(Fsn['stover']*FRAC_gasf+Fon['stover']*FRAC_gasm)
                    soil_N2O_feed_adn_rd['maize_bran']=maize_as_bran_area_rd*allocation_factor_maize_bran*EF_4_adn_rd*(Fsn['maize_bran']*FRAC_gasf+Fon['maize_bran']*FRAC_gasm)
                    soil_N2O_feed_adn_rd['sunflower_seed_meal']=sunflower_as_meal_area_rd*allocation_factor_sunflower_seed_meal*EF_4_adn_rd*(Fsn['sunflower_seed_meal']*FRAC_gasf+Fon['sunflower_seed_meal']*FRAC_gasm)
              
                # indirect (leaching) (kg n2o-n/yr)
                    soil_N2O_feed_lch_rd['pasture']=pasture_area_rd*EF_5_lch_rd*(Fsn['pasture']+Fcr['pasture']+Fon['pasture'])
                    soil_N2O_feed_lch_rd['grass']=grass_area_rd*EF_5_lch_rd*(Fsn['grass']+Fcr['grass']+Fon['grass']+Fprp['grass'])
                    soil_N2O_feed_lch_rd['napier']=napier_area_rd*EF_5_lch_rd*(Fsn['napier']+ Fon['napier']+Fcr['napier'])
                    soil_N2O_feed_lch_rd['napier_hy']=napier_hy_area_rd*EF_5_lch_rd*(Fsn['napier_hy']+ Fon['napier_hy']+Fcr['napier_hy'])
                    soil_N2O_feed_lch_rd['stover']=stover_area_rd*allocation_factor_stover*EF_5_lch_rd*(Fsn['stover']+ Fon['stover']+Fcr['stover'])
                    soil_N2O_feed_lch_rd['maize_bran']=maize_as_bran_area_rd*allocation_factor_maize_bran*EF_5_lch_rd*(Fsn['maize_bran']+Fcr['maize_bran'])
                    soil_N2O_feed_lch_rd['sunflower_seed_meal']=sunflower_as_meal_area_rd*allocation_factor_sunflower_seed_meal*EF_5_lch_rd*(Fsn['sunflower_seed_meal']+Fcr['sunflower_seed_meal'])
               

                    soil_N2O_feed_grass_rd=(soil_N2O_feed_direct_rd['grass']+soil_N2O_feed_lch_rd['grass']+soil_N2O_feed_adn_rd['grass'])
                    soil_N2O_feed_napier_rd=(soil_N2O_feed_direct_rd['napier']+soil_N2O_feed_lch_rd['napier']+soil_N2O_feed_adn_rd['napier'])
                    soil_N2O_feed_napier_hy_rd=(soil_N2O_feed_direct_rd['napier_hy']+soil_N2O_feed_lch_rd['napier_hy']+soil_N2O_feed_adn_rd['napier_hy'])
                    soil_N2O_feed_maize_bran_rd=(soil_N2O_feed_direct_rd['maize_bran']+soil_N2O_feed_lch_rd['maize_bran']+soil_N2O_feed_adn_rd['maize_bran'])
                    soil_N2O_feed_sunflower_seed_meal_rd=(soil_N2O_feed_direct_rd['sunflower_seed_meal']+soil_N2O_feed_lch_rd['sunflower_seed_meal']+soil_N2O_feed_adn_rd['sunflower_seed_meal'])
                    soil_N2O_feed_stover_rd=(soil_N2O_feed_direct_rd['stover']+soil_N2O_feed_lch_rd['stover']+soil_N2O_feed_adn_rd['stover'])
                    soil_N2O_feed_pasture_rd=(soil_N2O_feed_direct_rd['pasture']+soil_N2O_feed_lch_rd['pasture']+soil_N2O_feed_adn_rd['pasture'])

                    enteric_CH4_rd[r,ss,b,c]=alloc_factor*28*gross_energy[ss,b,c]*Ym_rd[ss,b,c]*(1/(100*55.65))
                    manure_CH4_rd[r,ss,b,c]=alloc_factor*(28*VS[ss,b,c]*B0_rd[ss,b,c]*MCF_rd[ss,b,c]*0.67)
                    manure_N2O_direct_rd[r,ss,b,c]=alloc_factor*265*(44/28)*FRAC_MANURE_MMT*((1/1000)*faecal_Nitrogen[ss,b,c]*EF_3_FAECAL_rd[ss,b,c]+(1/1000)*urinary_Nitrogen[ss,b,c]*EF_3_URINARY_rd[ss,b,c])
                    manure_N2O_leach_rd[r,ss,b,c]=alloc_factor*265*(44/28)*FRAC_MANURE_MMT*((1/1000)*(faecal_Nitrogen[ss,b,c]+urinary_Nitrogen[ss,b,c]))*FRAC_leach_rd*EF_5_rd
                    manure_N2O_vol_rd[r,ss,b,c]=alloc_factor*265*(44/28)*((1/1000)*FRAC_MANURE_MMT*(faecal_Nitrogen[ss,b,c]+urinary_Nitrogen[ss,b,c]))*FRAC_vol_rd*EF_4_rd
                    manure_N2O_rd[r,ss,b,c]=alloc_factor*manure_N2O_direct_rd[r,ss,b,c]+manure_N2O_leach_rd[r,ss,b,c]+manure_N2O_vol_rd[r,ss,b,c]
                    soil_N2O_rd[r,ss,b,c]=alloc_factor*265*(44/28)*(soil_N2O_feed_grass_rd*grass_area_rd+soil_N2O_feed_napier_rd*napier_area_rd+soil_N2O_feed_maize_bran_rd*maize_as_bran_area_rd+soil_N2O_feed_sunflower_seed_meal_rd*sunflower_as_meal_area_rd +soil_N2O_feed_stover_rd*stover_area_rd+soil_N2O_feed_pasture_rd*pasture_area_rd)
                    energy_use_CO2_rd[r,ss,b,c]=alloc_factor*energy_use_CO2[ss,b,c]*random_var[r,25]

                    grassland_area_expansion_emissions_rd_ss[r,ss]+=alloc_factor*random_var[r,20]*(44/12)*(herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*feed_primary_area_rd-herd_size_y_prev[(ss,b)]*herd_size[(ss,b,c)]*feed_primary_area_rd)*(luc_emission_coefficient_total_area)*(timestep/amort_period)/timestep 
                    cropland_area_expansion_emissions_rd_ss[r,ss]+=alloc_factor*random_var[r,21]*(44/12)*(herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*(crop_primary_area_rd)-herd_size_y_prev[(ss,b)]*herd_size[(ss,b,c)]*(crop_primary_area_rd))*(luc_emission_coefficient_grass_to_crop)*(timestep/amort_period)/timestep 


                   




        # report final results for this simulation
    for ss in subsectors:
        for b in breed:
            if (herd_size[(ss,b,'herd_pop')] ==0):
                
                continue
                
            # results are grouped numerically from v1 to v9, with each grouping for a specific type of result variable
            # This is necessary for the pandas dataframe to keep the results in order
            
            # v1 -- main production/productivity results
            # v2 -- nutrient intakes and N excretion
            # v3 -- feed intakes, simulation variables 
            # v4 -- animal specific emissions
            # v5 -- feed emission properties
            # v6 -- aggregated emissions 
            # v7 -- aggregated emissions intensity
            # v8 -- dairy herd populations
            # v9 -- land footprint
            
            for c in cohort:
                if (c == 'cow'):
                    
                    RESULTS['v1_1_Milk_yield_ss'+str(ss)+'_'+str(b)]=milk_yield_reg[ss,b]
                    RESULTS['v1_1_Milk_yield_lactation_kg_per_lactation__ss'+str(ss)+'_'+str(b)]=milk_yield_lact[ss,b]
                    RESULTS['v1_2_Milk_yield_total']+= herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,'cow')]*milk_yield_reg[ss,b]
                    RESULTS['v1_2_Milk_yield_base_year_kg_yr']+=herd_size_y_prev[(ss,b)]*herd_size[(ss,b,'cow')]*milk_yield_reg[ss,b]
                
                # results per breed and cohort and subsector
                RESULTS['v2_1_Dry_matter_intake_kg_per_head_per_yr_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=DMI[ss,b,c]
                RESULTS['v2_2_Dry_matter_intake_kg_per_head_per_day_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=DMI[ss,b,c]/365
                RESULTS['v2_3_Fraction_Roughage_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=fraction_roughage_eaten[ss,b,c]
                RESULTS['v2_3_Fraction_Concentrate_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=fraction_conc_eaten[ss,b,c]
                RESULTS['v2_3_Metabolisable_Energy_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=ME[ss,b,c]
                RESULTS['v2_4_Dry_matter_digestibility_%_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=DMD[ss,b,c]
                RESULTS['v2_5_1_Gross_energy_MJ_per_head_per_yr_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=gross_energy[ss,b,c]
                RESULTS['v2_5_2_Gross_energy_MJ_per_head_per_day_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=gross_energy[ss,b,c]/365
                RESULTS['v2_6_faecal_nitrogen_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=faecal_Nitrogen[ss,b,c]
                RESULTS['v2_6_urinary_nitrogen_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=urinary_Nitrogen[ss,b,c] 
                RESULTS['v2_7_Ym_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=Ym[ss,b,c]
                RESULTS['v2_8_VS_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=VS[ss,b,c]
                RESULTS['v2_9_Acid_detergent_fibre_g_per_head_per_year_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=ADF[ss,b,c]
                RESULTS['v2_9_Acid_detergent_fibre_%_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=100*ADF[ss,b,c]*(1/1000)/DMI[ss,b,c]
                RESULTS['v2_10_Neutral_detergent_fibre_g_per_head_per_year_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=NDF[ss,b,c]
                RESULTS['v2_10_Neutral_detergent_fibre_%_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=100*NDF[ss,b,c]*(1/1000)/DMI[ss,b,c]
                RESULTS['v2_11_Crude_protein_kg_per_head_per_year'+str(b)+'_'+str(c)+'_'+str(ss)+'']=crude_protein_intake[ss,b,c]
                RESULTS['v2_11_Crude_protein_%_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=100*crude_protein_intake[ss,b,c]/DMI[ss,b,c]
                
                # feed intake results (% dm for specific ss, b, c)
                RESULTS['v3_1_1_Feed_intake_grass_%_dm'+str(b)+'_'+str(c)+'_'+str(ss)+'']=100*grass_intake[ss,b,c]/DMI[ss,b,c]
                RESULTS['v3_1_2_Feed_intake_stover_%_dm'+str(b)+'_'+str(c)+'_'+str(ss)+'']=100*stover_intake[ss,b,c]/DMI[ss,b,c]
                RESULTS['v3_1_3_Feed_intake_napier_%_dm'+str(b)+'_'+str(c)+'_'+str(ss)+'']=100*napier_intake[ss,b,c]/DMI[ss,b,c]
                RESULTS['v3_1_4_Feed_intake_maize_bran_%_dm'+str(b)+'_'+str(c)+'_'+str(ss)+'']=100*maize_bran_intake[ss,b,c]/DMI[ss,b,c]
                RESULTS['v3_1_5_Feed_intake_sfsm_%_dm'+str(b)+'_'+str(c)+'_'+str(ss)+'']=100*sfsm_intake[ss,b,c]/DMI[ss,b,c]
                RESULTS['v3_1_6_Feed_intake_pasture_%_dm'+str(b)+'_'+str(c)+'_'+str(ss)+'']=100*pasture_intake[ss,b,c]/DMI[ss,b,c]

                # feed intake results (kg dm for entire herd)
                RESULTS['v3_2_6_Feed_intake_grass_kg_dm']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*grass_intake[ss,b,c]
                RESULTS['v3_2_7_Feed_intake_stover_kg_dm']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*stover_intake[ss,b,c]
                RESULTS['v3_2_8_Feed_intake_napier_kg_dm']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*napier_intake[ss,b,c]
                RESULTS['v3_2_8_Feed_intake_napier_hy_kg_dm']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*napier_hy_intake[ss,b,c]
                RESULTS['v3_2_9_Feed_intake_maize_bran_kg_dm']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*maize_bran_intake[ss,b,c]
                RESULTS['v3_2_10_Feed_intake_sfsm_kg_dm']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*sfsm_intake[ss,b,c]
                RESULTS['v3_2_11_Feed_intake_pasture_kg_dm']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*pasture_intake[ss,b,c]

                
                # seasonality results
                if (c == 'cow'):
                
                    
                    for i in range(0,cmc+1):
                        # Met energy intake in month i
                        RESULTS['v3_2_12_M_'+str(i)+'_metenergy_Mj_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=mo_dp_me[ss,b,c,i]/(count_calendar_month[i,ss,b,c])
                        
                        # Crude protein intake in month i
                        RESULTS['v3_2_12_M_'+str(i)+'_cp_g_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=mo_dp_cp[ss,b,c,i]/(count_calendar_month[i,ss,b,c])
            
                        # Shortage of MP in month i 
                        RESULTS['v3_2_12_M'+str(i)+'_MPS_g_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']= mo_mp_short[ss,b,c,i]/(count_calendar_month[i,ss,b,c])
                   
                         # Shortage of ME in month i 
                        RESULTS['v3_2_12_M'+str(i)+'_MES_Mj_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']= mo_me_short[ss,b,c,i]/(count_calendar_month[i,ss,b,c])
                   
                        
                   
             # average protein or energy deficit during dry season
                    RESULTS['v3_3_mean_dryse_prot_deficit']=(
                        RESULTS['v3_2_12_M1_MPS_g_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']
                        +RESULTS['v3_2_12_M2_MPS_g_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']
                        +RESULTS['v3_2_12_M3_MPS_g_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']
                        +RESULTS['v3_2_12_M4_MPS_g_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']
                        +RESULTS['v3_2_12_M5_MPS_g_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']
                        +RESULTS['v3_2_12_M6_MPS_g_d_'+str(b)+'_'+str(c)+'_'+str(ss)+''])*(1/6)
    
                    RESULTS['v3_3_mean_dryse_ene_deficit']=(
                        RESULTS['v3_2_12_M1_MES_Mj_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']+
                        RESULTS['v3_2_12_M2_MES_Mj_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']+
                        RESULTS['v3_2_12_M3_MES_Mj_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']+
                        RESULTS['v3_2_12_M4_MES_Mj_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']+
                        RESULTS['v3_2_12_M5_MES_Mj_d_'+str(b)+'_'+str(c)+'_'+str(ss)+'']+
                        RESULTS['v3_2_12_M6_MES_Mj_d_'+str(b)+'_'+str(c)+'_'+str(ss)+''])*(1/6)
                     
                
                
                # emissions per animal per year
                RESULTS['v4_1_Enteric_CH4_kg_CO2eq_per_head_per_year_'+str(ss)+'_'+str(b)+'_'+str(c)+'']=enteric_CH4[ss,b,c]
                RESULTS['v4_2_Manure_CH4_kg_CO2eq_per_head_per_year_'+str(ss)+'_'+str(b)+'_'+str(c)+'']=manure_CH4[ss,b,c] 
                RESULTS['v4_3_Manure_N2O_kg_CO2eq_per_head_per_year_'+str(ss)+'_'+str(b)+'_'+str(c)+'']=manure_N2O[ss,b,c] 
                RESULTS['v4_4_Soil_N2O_cropland_kg_CO2eq_per_head_per_year_'+str(ss)+'_'+str(b)+'_'+str(c)+''] =soil_N2O_cropland[ss,b,c] 
                RESULTS['v4_4_Soil_N2O_grassland_kg_CO2eq_per_head_per_year_'+str(ss)+'_'+str(b)+'_'+str(c)+''] =soil_N2O_grassland[ss,b,c] 
                RESULTS['v4_5_Feed_CO2_kg_CO2eq_per_head_per_year_'+str(ss)+'_'+str(b)+'_'+str(c)+''] =energy_use_CO2[ss,b,c]

                # emissions from soils
                RESULTS['v5_Grasslands_N2O_kg_per_ha_per_yr_ss_'+str(ss)+'']=(44/28)*soil_N2O_feed['grass']/grass_area[(ss,b,c)]
                RESULTS['v5_Napier_N2O_kg_per_ha_per_yr_ss_'+str(ss)+'']=(44/28)*soil_N2O_feed['napier']/napier_area[(ss,b,c)]
                RESULTS['v5_Maize_bran_N2O_kg_per_ha_per_yr_ss_'+str(ss)+'']=(44/28)*(1/allocation_factor_maize_bran)*soil_N2O_feed['maize_bran']/maize_as_bran_area[(ss,b,c)]
                RESULTS['v5_Sunflower_cake_N2O_kg_per_ha_per_yr_ss_'+str(ss)+'']=(44/28)*(1/allocation_factor_sunflower_seed_meal)*soil_N2O_feed['sunflower_seed_meal']/sunflower_as_meal_area[(ss,b,c)]
                RESULTS['v5_stover_N2O_kg_per_ha_per_yr_ss_'+str(ss)+'']=(44/28)*(1/allocation_factor_stover)*soil_N2O_feed['stover']/stover_area[(ss,b,c)]
                
                
                if (pasture_area[(ss,b,c)] !=0):
                    RESULTS['v5_Pasture_N2O_kg_per_ha_per_yr_ss_'+str(ss)+'']=(44/28)*soil_N2O_feed['pasture']/pasture_area[(ss,b,c)]
 
                # total area of feed land
                RESULTS['v5_6_Total_feed_primary_area_expansion']+=herd_size[(ss,b,c)]*(herd_size[(ss,b,'herd_pop')]*feed_primary_area[(ss,b,c)]-herd_size_y_prev[(ss,b)]*feed_primary_area[(ss,b,c)])

                
                ##  emissions results aggregated over breed and cohort for each lps
                RESULTS['v6_1_Enteric_CH4_aggregate_Mg_CO2eq_per_lps']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*enteric_CH4[ss,b,c]/1000 
                RESULTS['v6_2_Manure_CH4_aggregate_Mg_CO2eq_per_lps']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*manure_CH4[ss,b,c]/1000 
                RESULTS['v6_3_Manure_N2O_aggregate_Mg_CO2eq_per_lps']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*manure_N2O[ss,b,c]/1000 
                RESULTS['v6_4_Soil_N2O_cropland_aggregate_Mg_CO2eq_per_lps']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*soil_N2O_cropland[ss,b,c]/1000 
                RESULTS['v6_4_Soil_N2O_grassland_aggregate_Mg_CO2eq_per_lps']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*soil_N2O_grassland[ss,b,c]/1000 
                RESULTS['v6_4_2_Feed_CO2_aggregate_Mg_CO2eq_per_lps']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*energy_use_CO2[ss,b,c]/1000 


                
                # luc as part of aLCA

                RESULTS['v6_8_cropland_expansion_emissions_Mg_CO2eq_per_lps']+=(44/12)*herd_size[(ss,b,c)]*(herd_size[(ss,b,'herd_pop')]*crop_primary_area[(ss,b,c)]-herd_size_y_prev[(ss,b)]*crop_primary_area[(ss,b,c)])*(luc_emission_coefficient_grass_to_crop)*(timestep/amort_period)/timestep

                RESULTS['v6_5_Direct_emissions_aggregate_Mg_CO2eq_per_lps']=RESULTS['v6_1_Enteric_CH4_aggregate_Mg_CO2eq_per_lps']+RESULTS['v6_2_Manure_CH4_aggregate_Mg_CO2eq_per_lps']+RESULTS['v6_3_Manure_N2O_aggregate_Mg_CO2eq_per_lps']+RESULTS['v6_4_Soil_N2O_grassland_aggregate_Mg_CO2eq_per_lps']+RESULTS['v6_4_Soil_N2O_cropland_aggregate_Mg_CO2eq_per_lps']+RESULTS['v6_4_2_Feed_CO2_aggregate_Mg_CO2eq_per_lps']


            
                # herd populations
                RESULTS['v8_1_Herd_size_'+str(ss)+'_'+str(b)+'_'+str(c)]=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]
                RESULTS['v8_2_Herd_size_previous_year_'+str(ss)+'_'+str(b)+'_'+str(c)]=herd_size_y_prev[(ss,b)]*herd_size[(ss,b,c)]
                RESULTS['v8_3_Herd_size_base_'+str(ss)+'_'+str(b)+'_'+str(c)]=herd_size_base[(l,ss,b,'herd_pop')]*herd_size_base[(l,ss,b,c)]
                RESULTS['v8_4_Herd_size_total_1000_head_per_lps']+=herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]/1000                
                RESULTS['v8_6_Herd_size_total_TLU_per_lps']+=herd_size[(ss,b,c)]*herd_size[(ss,b,'herd_pop')]*TLU_equiv[(b,c)]
                RESULTS['v8_5_Herd_size_total_Million_TLU_per_lps']=RESULTS['v8_6_Herd_size_total_TLU_per_lps']/1000000
                
                # feed areas aggregated categories 
                RESULTS['v9_1_1_Grasslands_area_ha_per_head_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=total_grasslands_area[(ss,b,c)]
                RESULTS['v9_1_2_Feed_area_ha_per_head_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=feed_area[(ss,b,c)]
                RESULTS['v9_2_1_Grasslands_area_ha_per_lps'] += herd_size[(ss,b,c)]*(herd_size[(ss,b,'herd_pop')]*total_grasslands_area[(ss,b,c)])
                RESULTS['v9_2_2_Feed_area_ha_per_lps'] +=  herd_size[(ss,b,c)]*(herd_size[(ss,b,'herd_pop')]*(feed_area[(ss,b,c)]))
                RESULTS['v9_2_3_Crop_primary_area_ha_per_lps']+=herd_size[(ss,b,c)]*(herd_size[(ss,b,'herd_pop')]*(feed_area[(ss,b,c)]-total_grasslands_area[(ss,b,c)]))
                   
                # feed areas individual (ha per head)
                RESULTS['v9_3_1_Grass_area_ha_per_head_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=grass_area[(ss,b,c)]
                RESULTS['v9_3_2_Stover_area_ha_per_head_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=stover_area[(ss,b,c)]
                RESULTS['v9_3_3_Napier_area_ha_per_head_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=napier_area[(ss,b,c)]
                RESULTS['v9_3_4_Napier_hy_area_ha_per_head_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=napier_hy_area[(ss,b,c)]
                RESULTS['v9_3_5_Sunflower_area_ha_per_head_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=sunflower_as_meal_area[(ss,b,c)]
                RESULTS['v9_3_6_Maize_area_ha_per_head_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=maize_as_bran_area[(ss,b,c)]
                RESULTS['v9_3_7_Pasture_area_ha_per_head_'+str(b)+'_'+str(c)+'_'+str(ss)+'']=pasture_area[(ss,b,c)]
    
                # feed areas  (ha per lps)
                RESULTS['v9_4_1_Grass_area_ha_per_lps']+=grass_area[(ss,b,c)]*herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]
                RESULTS['v9_4_2_Stover_area_ha_per_lps']+=stover_area[(ss,b,c)]*herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]
                RESULTS['v9_4_3_Napier_area_ha_per_lps']+=napier_area[(ss,b,c)]*herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]
                RESULTS['v9_4_4_Napier_hy_area_ha_per_lps']+=napier_hy_area[(ss,b,c)]*herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]
                RESULTS['v9_4_5_Sunflower_area_ha_per_lps']+=sunflower_as_meal_area[(ss,b,c)]*herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]
                RESULTS['v9_4_6_Maize_area_ha_per_lps']+=maize_as_bran_area[(ss,b,c)]*herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]
                RESULTS['v9_4_7_Pasture_area_ha_per_lps']+=pasture_area[(ss,b,c)]*herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]
                RESULTS['v9_4_8_Total_area_ha_per_lps']=RESULTS['v9_4_1_Grass_area_ha_per_lps']+RESULTS['v9_4_2_Stover_area_ha_per_lps']+RESULTS['v9_4_3_Napier_area_ha_per_lps']+RESULTS['v9_4_4_Napier_hy_area_ha_per_lps']+RESULTS['v9_4_5_Sunflower_area_ha_per_lps']+RESULTS['v9_4_6_Maize_area_ha_per_lps']                
                
                
                

    
    ########### calculate biophysical feasibility of scenario #########
    ##### load pixel level data from excel         #########

    if (check_feasibility== 1):
        RESULTS['v6_0_total_native_ecosystems_converted(ha)']=0
        RESULTS['v6_0_total_grasslands_now_utilized(ha)']=0
        RESULTS['v6_0_Units_limited']=0
    
        # load data on spatial land use, lives populations
        system_feasibility_data=(cwd+str('\\land_feasibility.xlsx'))
        system_feasibility=xlrd.open_workbook(system_feasibility_data)
    
        if (l == 'MRT'):
            feas_sheet = system_feasibility.sheet_by_index(0)
        elif (l == 'MRH'):
            feas_sheet = system_feasibility.sheet_by_index(1)

        # load parameters from excel
        density={} # lives density per pixel (h/pixel)
        dgl_available={} #  grassland available as percentage of pixel
        frac_pop={} # the fraction of livestock in a pixel relative to total in the lps
        
        u=feas_sheet.nrows # number of rows in excel sheet

        total_popn=0 # total population of cattle in lps (head)
        
        for i in range(1,u): 
            total_popn+=feas_sheet.cell_value(i,2)

        for i in range(1,u): 
            
            dgl_available[i]=feas_sheet.cell_value(i,1)
            frac_pop[i]=feas_sheet.cell_value(i,2)/total_popn
            
        
        # total expansion of primary feed area for this scenario (ha/lps)
        total_expansion=RESULTS['v5_6_Total_feed_primary_area_expansion']
        area_growth={} # amount of new land needed for conversion per pixel (sq km/unit)
        
        for i in range(1,u):
            
            # amount of new land needed for primary feed expansion (weighted by cattle pop'n) (sq km)
            # pixel is 10km x 10km = 100 sq km 
            area_growth[i]=frac_pop[i]*total_expansion*.01   # (sq km per pixel)
            
            # if amount of new land needed (ha/100 sq km) is less than available grassland to be utilized (ha) (feas_sheet.cell_value(i,1))
            
            if (area_growth[i] <= dgl_available[i]):
                
                RESULTS['v6_0_total_grasslands_now_utilized(ha)']+=area_growth[i]
                
            # if amount of new land needed is greater than available grassland to be utilized
            elif (area_growth[i] > dgl_available[i]):
                
                RESULTS['v6_0_total_native_ecosystems_converted(ha)']+=(area_growth[i]-dgl_available[i])
                RESULTS['v6_0_total_grasslands_now_utilized(ha)']+=dgl_available[i]
                RESULTS['v6_0_Units_limited']+=1
            
        fraction_feed_primary_area_expansion_converting_native_ecosystems=(RESULTS['v6_0_total_native_ecosystems_converted(ha)']/(RESULTS['v6_0_total_grasslands_now_utilized(ha)']+RESULTS['v6_0_total_native_ecosystems_converted(ha)']))
            
        RESULTS['v6_6_grassland_expansion_emissions_Mg_CO2eq_per_lps']=fraction_feed_primary_area_expansion_converting_native_ecosystems*(44/12)*RESULTS['v5_6_Total_feed_primary_area_expansion']*(luc_emission_coefficient_total_area)*(timestep/amort_period)/timestep
                    
        
    
    
    # direct non co2 emissions
    RESULTS['v7_1_Enteric_CH4_intensity_kg_CO2eq_per_kg_milk_per_lps']=1000*RESULTS['v6_1_Enteric_CH4_aggregate_Mg_CO2eq_per_lps']/RESULTS['v1_2_Milk_yield_total'] 
    RESULTS['v7_2_Manure_CH4_intensity_kg_CO2eq_per_kg_milk_per_lps']=1000*RESULTS['v6_2_Manure_CH4_aggregate_Mg_CO2eq_per_lps']/RESULTS['v1_2_Milk_yield_total']
    RESULTS['v7_3_Manure_N2O_intensity_kg_CO2eq_per_kg_milk_per_lps']=1000*RESULTS['v6_3_Manure_N2O_aggregate_Mg_CO2eq_per_lps']/RESULTS['v1_2_Milk_yield_total']
    RESULTS['v7_4_Soil_N2O_cropland_intensity_kg_CO2eq_per_kg_milk_per_lps']=1000*RESULTS['v6_4_Soil_N2O_cropland_aggregate_Mg_CO2eq_per_lps']/RESULTS['v1_2_Milk_yield_total']
    RESULTS['v7_4_Soil_N2O_grassland_intensity_kg_CO2eq_per_kg_milk_per_lps']=1000*RESULTS['v6_4_Soil_N2O_grassland_aggregate_Mg_CO2eq_per_lps']/RESULTS['v1_2_Milk_yield_total']
    RESULTS['v7_4_Feed_CO2_intensity_kg_CO2eq_per_kg_milk_per_lps']=1000*RESULTS['v6_4_2_Feed_CO2_aggregate_Mg_CO2eq_per_lps']/RESULTS['v1_2_Milk_yield_total']
   
    RESULTS['v7_5_Direct emissions intensity_kg_CO2eq_per_kg_milk_per_lps']=RESULTS['v7_1_Enteric_CH4_intensity_kg_CO2eq_per_kg_milk_per_lps']+RESULTS['v7_2_Manure_CH4_intensity_kg_CO2eq_per_kg_milk_per_lps']+RESULTS['v7_3_Manure_N2O_intensity_kg_CO2eq_per_kg_milk_per_lps']+RESULTS['v7_4_Soil_N2O_cropland_intensity_kg_CO2eq_per_kg_milk_per_lps']+RESULTS['v7_4_Soil_N2O_grassland_intensity_kg_CO2eq_per_kg_milk_per_lps']+RESULTS['v7_4_Feed_CO2_intensity_kg_CO2eq_per_kg_milk_per_lps']

    
    # luc emissions
    RESULTS['v7_6_grassland_expansion_emissions_intensity_kg_CO2eq_per_kg_milk_per_lps']=1000*RESULTS['v6_6_grassland_expansion_emissions_Mg_CO2eq_per_lps']/RESULTS['v1_2_Milk_yield_total']
    RESULTS['v7_8_cropland_expansion_emissions_intensity_kg_CO2eq_per_kg_milk_per_lps']=1000*RESULTS['v6_8_cropland_expansion_emissions_Mg_CO2eq_per_lps']/RESULTS['v1_2_Milk_yield_total']
    RESULTS['v7_9_Indirect_emissions_intensity_kg_CO2eq_per_kg_milk_per_lps']=1000*(RESULTS['v6_6_grassland_expansion_emissions_Mg_CO2eq_per_lps']+RESULTS['v6_8_cropland_expansion_emissions_Mg_CO2eq_per_lps'])/RESULTS['v1_2_Milk_yield_total']
    RESULTS['v7_5_Total emissions intensity_kg_CO2eq_per_kg_milk_per_lps']=RESULTS['v7_5_Direct emissions intensity_kg_CO2eq_per_kg_milk_per_lps']+RESULTS['v7_9_Indirect_emissions_intensity_kg_CO2eq_per_kg_milk_per_lps']
    
    # total
    RESULTS['v7_10_Emissions_total_Mg_CO2eq_per_lps']=RESULTS['v6_5_Direct_emissions_aggregate_Mg_CO2eq_per_lps']+RESULTS['v6_6_grassland_expansion_emissions_Mg_CO2eq_per_lps']+RESULTS['v6_8_cropland_expansion_emissions_Mg_CO2eq_per_lps']
    
    # per tlu
    RESULTS['v7_11_Enteric_CH4_intensity_kg_CO2eq_per_TLU_per_lps']=1000*RESULTS['v6_1_Enteric_CH4_aggregate_Mg_CO2eq_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v7_12_Manure_CH4_intensity_kg_CO2eq_per_TLU_per_lps']=1000*RESULTS['v6_2_Manure_CH4_aggregate_Mg_CO2eq_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v7_13_Manure_N2O_intensity_kg_CO2eq_per_TLU_per_lps']=1000*RESULTS['v6_3_Manure_N2O_aggregate_Mg_CO2eq_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v7_14_Soil_N2O_cropland_intensity_kg_CO2eq_per_TLU_per_lps']=1000*RESULTS['v6_4_Soil_N2O_cropland_aggregate_Mg_CO2eq_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v7_14_Soil_N2O_grassland_intensity_kg_CO2eq_per_TLU_per_lps']=1000*RESULTS['v6_4_Soil_N2O_grassland_aggregate_Mg_CO2eq_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v7_14_Feed_CO2_intensity_kg_CO2eq_per_TLU_per_lps']=1000*RESULTS['v6_4_2_Feed_CO2_aggregate_Mg_CO2eq_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
   

    RESULTS['v7_16_grassland_expansion_emissions_intensity_kg_CO2eq_per_TLU_per_lps']=1000*RESULTS['v6_6_grassland_expansion_emissions_Mg_CO2eq_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v7_18_cropland_expansion_emissions_intensity_kg_CO2eq_per_TLU_per_lps']=1000*RESULTS['v6_8_cropland_expansion_emissions_Mg_CO2eq_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
   
   
    
    
    # multiply by 100 to convert to a percentage
    RESULTS['v1_5_Feed_intake_Mt_per_LPS']=(1/(1000*1000000))*(RESULTS['v3_2_6_Feed_intake_grass_kg_dm']+RESULTS['v3_2_7_Feed_intake_stover_kg_dm']+RESULTS['v3_2_8_Feed_intake_napier_kg_dm']+RESULTS['v3_2_9_Feed_intake_maize_bran_kg_dm']+RESULTS['v3_2_10_Feed_intake_sfsm_kg_dm']+RESULTS['v3_2_11_Feed_intake_pasture_kg_dm'])
    RESULTS['v3_2_1_Feed_intake_grass_(% dm)']=(100/(1000*1000000))*RESULTS['v3_2_6_Feed_intake_grass_kg_dm']/RESULTS['v1_5_Feed_intake_Mt_per_LPS']
    RESULTS['v3_2_2_Feed_intake_stover_(% dm)']=(100/(1000*1000000))*RESULTS['v3_2_7_Feed_intake_stover_kg_dm']/RESULTS['v1_5_Feed_intake_Mt_per_LPS']
    RESULTS['v3_2_3_Feed_intake_napier_(% dm)']=(100/(1000*1000000))*RESULTS['v3_2_8_Feed_intake_napier_kg_dm']/RESULTS['v1_5_Feed_intake_Mt_per_LPS']
    RESULTS['v3_2_3_Feed_intake_napier_hy_(% dm)']=(100/(1000*1000000))*RESULTS['v3_2_8_Feed_intake_napier_hy_kg_dm']/RESULTS['v1_5_Feed_intake_Mt_per_LPS']
    RESULTS['v3_2_4_Feed_intake_maize_bran_(% dm)']=(100/(1000*1000000))*RESULTS['v3_2_9_Feed_intake_maize_bran_kg_dm']/RESULTS['v1_5_Feed_intake_Mt_per_LPS']
    RESULTS['v3_2_5_Feed_intake_sfsm_(% dm)']=(100/(1000*1000000))*RESULTS['v3_2_10_Feed_intake_sfsm_kg_dm']/RESULTS['v1_5_Feed_intake_Mt_per_LPS']
    RESULTS['v3_2_6_Feed_intake_pasture_(% dm)']=(100/(1000*1000000))*RESULTS['v3_2_11_Feed_intake_pasture_kg_dm']/RESULTS['v1_5_Feed_intake_Mt_per_LPS']

    RESULTS['v3_3_1_Feed_intake_grass_kg_per_TLU']=(1/RESULTS['v8_6_Herd_size_total_TLU_per_lps'])*RESULTS['v3_2_6_Feed_intake_grass_kg_dm']
    RESULTS['v3_3_2_Feed_intake_stover_kg_per_TLU']=(1/RESULTS['v8_6_Herd_size_total_TLU_per_lps'])*RESULTS['v3_2_7_Feed_intake_stover_kg_dm']
    RESULTS['v3_3_3_Feed_intake_napier_kg_per_TLU']=(1/RESULTS['v8_6_Herd_size_total_TLU_per_lps'])*RESULTS['v3_2_8_Feed_intake_napier_kg_dm']
    RESULTS['v3_3_3_Feed_intake_napier_hy_kg_per_TLU']=(1/RESULTS['v8_6_Herd_size_total_TLU_per_lps'])*RESULTS['v3_2_8_Feed_intake_napier_hy_kg_dm']
    RESULTS['v3_3_4_Feed_intake_maize_bran_kg_per_TLU']=(1/RESULTS['v8_6_Herd_size_total_TLU_per_lps'])*RESULTS['v3_2_9_Feed_intake_maize_bran_kg_dm']
    RESULTS['v3_3_5_Feed_intake_sfsm_kg_per_TLU']=(1/RESULTS['v8_6_Herd_size_total_TLU_per_lps'])*RESULTS['v3_2_10_Feed_intake_sfsm_kg_dm']
    RESULTS['v3_3_6_Feed_intake_pasture_kg_per_TLU']=(1/RESULTS['v8_6_Herd_size_total_TLU_per_lps'])*RESULTS['v3_2_11_Feed_intake_pasture_kg_dm']

    # summation
    RESULTS['v9_15_Grass_land_use_ha_per_TLU']=RESULTS['v9_4_1_Grass_area_ha_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v9_16_Stover_land_use_ha_per_TLU']=RESULTS['v9_4_2_Stover_area_ha_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v9_17_Napier_land_use_ha_per_TLU']=RESULTS['v9_4_3_Napier_area_ha_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v9_17_Napier_hy_land_use_ha_per_TLU']=RESULTS['v9_4_4_Napier_hy_area_ha_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v9_18_Sunflower_land_use_ha_per_TLU']=RESULTS['v9_4_5_Sunflower_area_ha_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v9_19_Maize_land_use_ha_per_TLU']=RESULTS['v9_4_6_Maize_area_ha_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    RESULTS['v9_20_Pasture_land_use_ha_per_TLU']=RESULTS['v9_4_7_Pasture_area_ha_per_lps']/RESULTS['v8_6_Herd_size_total_TLU_per_lps']
    
    

                
    
    
    
    
    
    
    RESULTS['v1_3_Milk_productivity_kg_per_ha']= RESULTS['v1_2_Milk_yield_total']/(RESULTS['v9_4_8_Total_area_ha_per_lps'])
    RESULTS['v1_4_Land_footprint_Mha_per_LPS']=  (1/1000000)*(RESULTS['v9_4_8_Total_area_ha_per_lps'])
    
    RESULTS['v1_6_Feed_efficiency_kg_FPCM_per_Mg_feed_dm']=(1)*(1000/(1000*1000000))*RESULTS['v1_2_Milk_yield_total']/ RESULTS['v1_5_Feed_intake_Mt_per_LPS']
    
   
    # sort results alphabetically/alphanumerically
    sorted_list=sorted(RESULTS.keys(), key=lambda x:x.lower())

    results_ordered={}

    for i in sorted_list:
        results_ordered[i]=RESULTS[i]
    
    RESULTS={}
    RESULTS=results_ordered
    
    
    RESULTS['v_11_1_Emissions_total_Mg_CO2eq_per_lps']=RESULTS['v6_5_Direct_emissions_aggregate_Mg_CO2eq_per_lps']+(RESULTS['v6_8_cropland_expansion_emissions_Mg_CO2eq_per_lps']+RESULTS['v6_6_grassland_expansion_emissions_Mg_CO2eq_per_lps'])

           
    
   
    
    for r in range(0,MC_sims):
        direct_emission_absolute_rd[r]=0
        
        grassland_area_expansion_emissions_rd[r]=0
        cropland_area_expansion_emissions_rd[r]=0
                    
                    
        for ss in subsectors:
             
            for b in breed:
                if (herd_size[(ss,b,'herd_pop')] ==0):
                    continue
                for c in cohort:
                    direct_emission_absolute_rd[r]+= herd_size[(ss,b,'herd_pop')]*herd_size[(ss,b,c)]*(enteric_CH4_rd[r,ss,b,c]+manure_CH4_rd[r,ss,b,c]+manure_N2O_rd[r,ss,b,c]+soil_N2O_rd[r,ss,b,c]+energy_use_CO2_rd[r,ss,b,c])/1000
                    
        for ss in subsectors:    
            grassland_area_expansion_emissions_rd[r]+=grassland_area_expansion_emissions_rd_ss[r,ss]*fraction_feed_primary_area_expansion_converting_native_ecosystems
            cropland_area_expansion_emissions_rd[r]+=cropland_area_expansion_emissions_rd_ss[r,ss]
                    
                    
                    
        direct_emission_absolute_rd[r]+=(cropland_area_expansion_emissions_rd[r]+grassland_area_expansion_emissions_rd[r])

        
    variation_ab_squared=list(range(0, MC_sims))
    total_ab_var=list(range(0, MC_sims))
    total_ab=0
    location_start_error=len(RESULTS)-MC_sims
    total_ab_variation=0

        
    for r in range(0,MC_sims):
        variation_ab_squared[r]=(direct_emission_absolute_rd[r]-RESULTS['v_11_1_Emissions_total_Mg_CO2eq_per_lps'])**2
            
            
        total_ab_variation+=variation_ab_squared[r]
            
        
    total_ab=math.sqrt(total_ab_variation/MC_sims)
    ninetyfivepci=1.96*total_ab
    RESULTS['v_11_2_error_ab']=ninetyfivepci/1000000
    RESULTS['v_11_2_error_int']=(1000)*ninetyfivepci/RESULTS['v1_2_Milk_yield_total']
    RESULTS['v_11_2_error_per_tlu']=(1000)*ninetyfivepci/RESULTS['v8_6_Herd_size_total_TLU_per_lps']

        
        
    return RESULTS
                            